# Assignment 2: Recurrent neural networks

## Due: by 11:59pm, Wednesday 6/7

*Note: late submissions are accepted up to 5 days after the deadline for this assignment. You will receive 4 free late days to use between the two assignments of the course; once you exhaust those late days, each (partial) day that a submission is late will cap your maximum grade for the assignment by 10%. If you submit this assignment more than 5 days late, you will receive no credit.*

**Before submitting**, please click the *Kernel* menu at the top of JupyterLab and select the *Restart Kernel and Run All Cells...* button, then click the red *Restart* button on the box that pops up. This ensures that your code will run on its own, without relying on information that you may have added and then removed when developing it. **Please note that some parts of the code may be time-consuming to run. You should plan accordingly so that you don't start rerunning the code too close to the deadline.**

**To submit** your assignment, first make sure you have saved it, by pressing *Ctrl + S* or *Cmd + S* or clicking on the disk icon at the top of the notebook. Then [use git / GitHub Desktop](https://gauchospace.ucsb.edu/courses/mod/page/view.php?id=4033867) to *add* your completed assignment file(s) to git tracking, *commit* them to your repository with the message `Final submission`, and *push* them to GitHub. **We recommend going through the process of submitting sufficiently early that you can come to office hours to get help if needed; you can submit multiple times, and only your final submission (with the commit message `Final submission` will be graded).*

## Please answer this question prior to submitting

<span style="color: red;">**Did you consult anyone other than the TA/instructor, or any resources other than those listed on GauchoSpace, for this assignment? If so, please list them below.**</span>

> It is fine to consult classmates and external resources, as long as the work you submit is your own. We would just like to know who you consulted, or what other resources you might have found helpful.

*Double-click on this text cell to enter your response*

# Part 1: RNNs in PyTorch [50 pts]

In this part of the assignment, you'll learn how to create recurrent neural networks in PyTorch, and you'll show your conceptual understanding by exploring and explaining key design decisions.

Here are some other tutorials that explore aspects of RNNs we aren't able to fit in this assignment:  
- [NLP From Scratch: Classifying Names with a Character-Level RNN](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)  
- [Generating spam from a character-level RNN](https://www.cs.toronto.edu/~lczhang/aps360_20191/lec/w08/rnn.html)  
- [NLP From Scratch: Generating Names with a Character-Level RNN](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html)  
- [Seq2seq learning with neural networks](https://github.com/bentrevett/pytorch-seq2seq/tree/master)  
- [NLP From Scratch: Translation with a Sequence to Sequence Network and Attention](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)  

To begin with, run the cell below to import everything you'll need. It will also create several functions and variables from Assignment 1 that we'll be reusing here.

In [ ]:
import torch
from torch import nn, optim, tensor
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vectors
import pandas as pd
import time

pd.set_option('display.max_colwidth', None)


# ========================
# LOADING DATA - GENERAL

class TSVDataset(Dataset):
    
    def __init__(self, filepath):
        """Loads the data from a provided filepath"""
        self.data = list()
        with open(filepath, encoding="utf-8") as in_file:
            for line in in_file:
                (label, text) = line.strip().split("\t")
                self.data.append((label, text))

    def __getitem__(self, idx):
        """Returns the datapoint at a given index"""
        return self.data[idx]
    
    def __len__(self):
        """Returns the number of datapoints in the dataset"""
        return len(self.data)

spacy_tokenizer = get_tokenizer('spacy', language="en_core_web_sm")
tokenizer = lambda text: [token.lower() for token in spacy_tokenizer(text)]
    
def text_to_indices(text):
    tokens = tokenizer(text)
    indices = vocab(tokens)
    return torch.tensor(indices, dtype=torch.int64)

def label_to_index(label):
    return int(label == "pos")

def data_to_indices(data):
    (label, text) = data
    return (label_to_index(label), text_to_indices(text))

train_data = TSVDataset("inputs/imdb-train.tsv")
test_data = TSVDataset("inputs/imdb-test.tsv")


# =================================================
# SETTING UP THE VOCAB AND EMBEDDINGS - GENERAL

def yield_tokens(data):
    """A generator for tokenizing text in a (label, text) pair"""
    for _, text in data:
        yield tokenizer(text)

tokenized_iter = yield_tokens(train_data)
embeddings = Vectors("inputs/glove_6B_50_sample_train.txt")


# ========================================
# TRAINING AND TESTING - GENERAL

def train(model, dataloader, optimizer, epochs=100, print_every=1,
          validation_data=None):
    """Train a PyTorch model and print results periodically
    
    Arguments
    ---------
    model: torch.nn.Module; the model to be trained
    dataloader: torch.utils.data.DataLoader; the training data
    optimizer: the PyTorch optimizer to use for training
    epochs: int; the number of complete cycles through the training data
    print_every: int; print the results after this many epochs
                 (does not print if this is None)
    validation_data: torch.utils.data.DataLoader; the validation data
    """
    start_time = time.time()
    
    if print_every is not None:
        # Print initial performance
        initial_performance = test(model, dataloader)
        log_message = '| epoch   0 | train acc {acc:6.3f} | train loss {loss:6.3f} |'.format(**initial_performance)
        if validation_data is not None:
            validation_performance = test(model, validation_data)
            log_message += ' valid acc {acc:6.3f} | valid loss {loss:6.3f} |'.format(**validation_performance)
        print(log_message)
        
        # Set up trackers for printing results along the way
        total_acc = 0
        total_count = 0
        current_loss = 0.0
        minibatches_per_log = len(dataloader) * print_every
        
    # Tell the model that these inputs will be used for training
    model.train()
        
    for epoch in range(epochs):
        # Within each epoch, iterate over the data in mini-batches
        # Note the use of *datapoint_list for generality, whether or not there are offsets
        for (label_list, *datapoint_list) in dataloader:
            
            # Clear out gradients accumulated from inputs in the previous mini-batch
            model.zero_grad()

            # Run the forward pass to make predictions for the mini-batch
            predicted_probs = model(*datapoint_list).view(-1)

            # Compute the loss and send it backward through the network to get gradients
            # Note: PyTorch averages the loss over all datapoints in the minibatch
            loss = model.loss_function(predicted_probs, label_list.to(torch.float32))
            loss.backward()
            
            # Nudge the weights
            optimizer.step()
            
            # Track performance
            if print_every is not None: 
                total_acc += ((predicted_probs > 0.5).to(torch.int64) == label_list).sum().item()
                total_count += label_list.size(0)
                current_loss += loss.item()

        # Log performance
        if print_every is not None and (epoch + 1) % print_every == 0:
            log_message = ('| epoch {:3d} | train acc {:6.3f} | train loss {:6.3f} |'
                           .format(epoch + 1, total_acc/total_count, current_loss/minibatches_per_log))
            if validation_data is not None:
                validation_performance = test(model, validation_data)
                log_message += ' valid acc {acc:6.3f} | valid loss {loss:6.3f} |'.format(**validation_performance)
            print(log_message)

            # Reset trackers after logging
            total_acc = 0
            total_count = 0
            current_loss = 0.0
            model.train()
            
    print("\nOverall training time: {:.0f} seconds".format(time.time() - start_time))
            
def test(model, dataloader):
    """Evaluate a PyTorch model by testing it on labeled data
    
    Arguments
    ---------
    model: torch.nn.Module; the model to be tested
    dataloader: torch.utils.data.DataLoader; the test data
    """
    # Tell the model that these inputs will be used for evaluation
    model.eval()
    
    # Set up trackers
    total_acc = 0
    total_count = 0
    loss = 0.0

    with torch.no_grad(): # This can speed things up by telling PyTorch to ignore gradients
        # Note the use of *datapoint_list for generality, whether or not there are offsets
        for (label_list, *datapoint_list) in dataloader:
            # Get the model's output predictions
            predicted_probs = model(*datapoint_list).view(-1)
            predicted_labels = (predicted_probs > 0.5).to(torch.int64)
            
            # Calculate the loss and accuracy
            loss += model.loss_function(predicted_probs, label_list.to(torch.float32)).item()
            total_acc += (predicted_labels == label_list).sum().item()
            total_count += label_list.size(0)
    
    performance = {"acc": total_acc/total_count, "loss": loss/len(dataloader)}
    return performance


# ==================================
# INSPECTING A MODEL - GENERAL

def display_weights(model):
    """Prints the weights of a model"""
    for name, param in model.named_parameters():
        print(name.upper(), param)
        print()
        
def predict_multiple(model, texts, collate_batch_fn, labels=["neg", "pos"]):
    """Prints a model's predictions for a list of input texts.
    
    Arguments
    ---------
    model: torch.nn.Module; a PyTorch RNN model
    texts: list(str); a list of untokenized strings to feed as input to the model
    collate_batch_fn: function; a function that is used to prepare (batched) data
                      to be input into the model
    labels: list(str); a list of the labels that correspond to the indices the
            model will output
    """
    # Tell the model not to use these inputs for training
    model.eval()
    
    # Convert the input texts to indices, and get other model arguments needed
    data = [(None, text) for text in texts]
    (_, *model_input) = collate_batch_fn(data)
    
    # Feed the inputs through the model
    with torch.no_grad():
        probs = model(*model_input).view(-1)
    
    # Collate the predictions in a DataFrame
    predictions = pd.DataFrame({"Input text": texts, "Classifier probability": probs})
    predictions["Output label"] = labels[0]
    predictions.loc[predictions["Classifier probability"] > 0.5, "Output label"] = "pos"
    return predictions

        
# =================================
# LOADING DATA - SPECIFIC TO BOE

def collate_batch_boe(batch):
    """Converts a batch of data into PyTorch tensor format, and collates
    the results by label, text, and offset, for use in a bag-of-embeddings
    model.
    """
    # Initialize lists that separate out the three components
    label_list = list()
    text_list = list()
    offsets_list = [0]
    
    for data in batch:
        # Convert to PyTorch format
        (label_index, text_indices) = data_to_indices(data)
        # Add converted data to separate component lists
        label_list.append(label_index)
        text_list.append(text_indices)
        offsets_list.append(text_indices.size(0))
        
    # Convert everything to tensors
    label_tensor = torch.tensor(label_list, dtype=torch.int64)
    text_tensor = torch.cat(text_list)
    offsets_tensor = torch.tensor(offsets_list[:-1]).cumsum(dim=0)
    
    return (label_tensor, text_tensor, offsets_tensor)
          
train_dataloader_boe = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_batch_boe)
test_dataloader_boe = DataLoader(test_data, batch_size=1000, collate_fn=collate_batch_boe)

## Activity 1.1: Comparing recurrent and bag-of-embeddings classifiers

In this activity, we'll construct sentiment analysis models for reviews in two ways: using a bag-of-embeddings approach, and using a recurrent neural network. 

The model that we'll use for a bag-of-embeddings approach is a simpler version of the `BagOfEmbeddingsBinaryClassifier` class from Assignment 1. This model forms a document embedding from the average of the embeddings for all the words in the document, then uses that document embedding in a feedforward neural network with a single hidden layer in order to predict whether the document has positive or negative sentiment.

The code cell below defines a class that can be used as a factory for the BoE model.

In [ ]:
class BagOfEmbeddingsBinaryClassifier(nn.Module):

    def __init__(self, vocab, embeddings, hidden_dim, freeze_embeddings=True):
        super(BagOfEmbeddingsBinaryClassifier, self).__init__()
        
        self.vocab = vocab
        
        vocab_embeddings = embeddings.get_vecs_by_tokens(self.vocab.get_itos())
        self.embedding = nn.EmbeddingBag.from_pretrained(vocab_embeddings, freeze=freeze_embeddings, mode="mean")
        
        # The hidden layer will go from the embeddings to a layer of hidden_dim units
        self.hidden_layer = nn.Sequential(
            nn.Linear(embeddings.dim, hidden_dim),
            nn.ReLU()
        )
        
        # The output layer will go from the hidden layer (hidden_dim units) to a single unit
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
        self.loss_function = nn.BCELoss()

    def forward(self, text, offsets):
        # The hidden layer slots in between the embeddings and the outputs
        doc_embedding = self.embedding(text, offsets)
        hidden = self.hidden_layer(doc_embedding)
        output = self.output_layer(hidden)
        return output

For the RNN, we'll use a unidirectional (forward) model that is as similar to the bag-of-embeddings model as possible. That means the model will take the sequence of word embeddings in the document as input, combine them into a document embedding using recurrence, and then use that document embedding in a feedforward neural network with a single hidden layer to make the prediction. We'll have all the layers be the same size as they are in the BoE model; that means the recurrent layer will be the same size as the input embeddings (so that the resultant document embedding is also the same size as the word embeddings, like it is in BoE), and the hidden layer in the classifier will be the same size as it is in BoE. 

The code cell below defines a class that can be used as a factory for the RNN model, using either `"tanh"` or `"relu"` as the `recurrent_activation` function. We will only use tanh (the default) in this assignment.

In [ ]:
class RNNClassifier(nn.Module):

    def __init__(self, vocab, embeddings, recurrent_dim, hidden_dim, freeze_embeddings=True,
                 recurrent_activation="tanh", recurrent_layers=1, recurrent_bidirectional=False):
        super(RNNClassifier, self).__init__()
        
        self.vocab = vocab
        
        vocab_embeddings = embeddings.get_vecs_by_tokens(self.vocab.get_itos())
        padding_idx = self.vocab.get_stoi().get("<pad>")  # Get the <pad> index
        self.embedding = nn.Embedding.from_pretrained(vocab_embeddings, freeze=freeze_embeddings, 
                                                      padding_idx=padding_idx) # Tell PyTorch that <pad> is for padding
        
        # The embeddings go into an RNN layer with recurrent_dim units
        self.recurrent_layer = nn.RNN(embeddings.dim, recurrent_dim, nonlinearity=recurrent_activation,
                                      num_layers=recurrent_layers, bidirectional=recurrent_bidirectional,
                                      batch_first=True) # Because we'll make the mini-batch a list of sequences
        
        # The recurrent output creates a doc_embedding, which feeds into a of hidden_dim units
        # We'll be concatenating the forward and backward direction of all layers
        # from the recurrent output, so the doc_embedding will be sized accordingly
        doc_embedding_dim = recurrent_dim * recurrent_layers * int(1 + recurrent_bidirectional)
        self.hidden_layer = nn.Sequential(
            nn.Linear(doc_embedding_dim, hidden_dim),
            nn.ReLU()
        )
        
        # The output layer will go from the hidden layer (hidden_dim units) to a single unit
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
        self.loss_function = nn.BCELoss()

    def forward(self, padded_text, seq_lengths):
        word_embeddings = self.embedding(padded_text)
        
        # The sequence of word embeddings has to be packed for efficiency of the RNN
        packed_word_embeddings = pack_padded_sequence(word_embeddings, seq_lengths, batch_first=True, enforce_sorted=False)
        (final_layer_all_timesteps, all_layers_final_timestep) = self.recurrent_layer(packed_word_embeddings)
        
        # all_layers_final_timestep contains the activations of all (stacked / bidirectional) recurrent
        # layers at the final timestep for each sequence (taking the padding into account).
        # For our classifier, we will stick all of these layers together (forward + backward, 
        # for each stacked layer) to use as the document embedding.
        # all_layers_final_timestep has shape (num_layers, minibatch_size, recurrent_dim);
        # we want something of shape (minibatch_size, num_layers * recurrent_dim),
        # so we reorder the dimensions and then reshape to stick everything together
        minibatch_size = all_layers_final_timestep.size(1)
        doc_embedding = all_layers_final_timestep.permute(1, 0, 2).reshape(minibatch_size, -1)
        
        hidden = self.hidden_layer(doc_embedding)
        output = self.output_layer(hidden)
        return output

All that differs between our two models is how they make the document embedding from the word embeddings: BoE uses (mean) pooling, while RNN uses recurrence. The BoE model can therefore be thought of as a special case of the RNN model, where there is no activation function at the recurrent layer, the input weights are fixed to $I/n$ (where $I$ is the identity matrix; this divides each word embedding by the number of words in the document, then adds it to the pool at the recurrent layer), and the recurrent weights are fixed to $I$ (where $I$ is the identity matrix; this carries forward all of the information pooled from the previous timesteps).

**MINI-BATCHING IN AN RNN**

Recall that we usually provide training samples to the model in *mini-batches*, for efficiency. All of the training samples in a mini-batch are processed together. For feedfoward NNs, the input in each training sample was a vector of the same size; but for RNNs, the input in each training sample is a *sequence* of vectors, and different sequences may have different lengths. In order to form mini-batches appropriately, we have to ensure that all of the sequences in a mini-batch are the same length.

Of course, we probably don't want to actually change the length of the input sequences in our training data, by cutting long sequences short. So instead, we *pad* short sequences to make them longer. PyTorch knows that this padding is only there for technical reasons, and shouldn't count when making predictions or doing backprop.

To be able to pad the sequences, we first have to add a special symbol `"<pad>"` to our vocabulary (like `"<unk>"`). The best time to do this is when loading the data, by including `"<pad>"` in the list of `specials` given as keyword argument to the `build_vocab_from_iterator()` function. The `specials` are represented by the first few indices in the vocab; for example, if `specials=["<pad>", "<unk>"]`, then `"<pad>"` will be represented by the index 0 and `"<unk>"` will be represented by the index 1:

In [ ]:
vocab = build_vocab_from_iterator(tokenized_iter, specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

padding_idx = vocab.get_stoi().get("<pad>")
print("The <pad> symbol in this vocabulary is represented by the index {}".format(padding_idx))

We also have to tell the input (`self.embedding`) layer what the padding index is, which is accomplished through the `padding_idx` keyword argument of the  `nn.Embedding.from_pretrained()` method in the `RNNClassifier` class above.

One we have a pad token in the vocab, we need to use it to pad out the mini-batch of sequences, and we need to tell PyTorch to ignore this padding when making predictions or doing backprop (which is accomplished by *packing* the sequence). PyTorch provides two functions to help with this: `pad_sequence()` and `pack_padded_sequence()` (both in `torch.nn.utils.rnn`). We use `pad_sequence()` in our custom `collate_batch()` function that we use to get mini-batches from a DataLoader, and `pack_padded_sequence()` in the `forward()` method of the `RNNClassifier` (because it is the *embeddings* that need to be packed, and they are looked up inside `forward()`).

In [ ]:
def collate_batch_rnn(batch):
    """Converts a batch of sequence data into padded and packed PyTorch 
    tensor format, and collates the results by label, text, and sequence
    length, for use in a RNN model.
    """
    # Initialize lists that separate out the two components
    label_list = list()
    text_list = list()
    seq_lengths = list()
    
    for data in batch:
        # Convert to PyTorch format
        (label_index, text_indices) = data_to_indices(data)
        # Add converted data to separate component lists
        label_list.append(label_index)
        text_list.append(text_indices)
        seq_lengths.append(len(text_indices))
    
    # Convert to mini-batch tensors
    label_tensor = torch.tensor(label_list, dtype=torch.int64)
    text_tensor = pad_sequence(text_list, batch_first=True, padding_value=padding_idx)
    
    return (label_tensor, text_tensor, seq_lengths)
          
train_dataloader_rnn = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_batch_rnn)
test_dataloader_rnn = DataLoader(test_data, batch_size=1000, collate_fn=collate_batch_rnn)

Note that padding complicates matters somewhat: we want the classifier to use the recurrent layer activations from the last non-padding timestep of each sequence, but they may all be in different positions! This is the reason for the extra steps taken to get `doc_embedding` inside `RNNClassifier.foward()`.

---
---
### Questions 1.1.1 - 1.1.2 

The code cell below shows an example of padding. Run it, then answer the following question:

In [ ]:
# Demonstration of padding
texts = ["I hated it", "it was quite terrible", "i really REALLY loved it"]
print("texts:\n{}\n".format(texts))

text_indices = [text_to_indices(text) for text in texts]
print("Converted to indices:\n{}\n".format(text_indices))

padded = pad_sequence(text_indices, batch_first=True, padding_value=padding_idx)
print("Padded:\n{}".format(padded))

**QUESTION 1.1.1. [1 point]**  
How do the padded sequences relate to the original sequences? What are the 0s in the padded tensor, why are they where they are, and why are there as many as there are?

*Double-click on this cell to enter your response.*

---

The code cell below demonstrates the importance of telling PyTorch to ignore padding, through use of *packing*. It defines a simple RNN in which the recurrent layer activation at each timestep increases by $1 + x_t$, where $x_t$ is the input value at timestep $t$. Thus, the recurrent layer activation at timestep $t$ is equal to $t$ plus the sum of the input values at all timesteps up to and including the current timestep.

Run the code cell, then answer the following question:

In [ ]:
with torch.no_grad(): # This tells PyTorch we aren't going to be doing backprop
    
    # Define simple embedding and recurrent layers.
    # The embedding layer just contains the index of the input
    # The recurrent layer has a bias of 1 and all weights fixed to 1,
    # so it adds together all of the inputs up to the current timestep,
    # plus the number of timesteps since the beginning of the sequence
    embedding_layer = nn.Embedding.from_pretrained(torch.arange(5000, dtype=torch.float32).view(-1, 1), 1, padding_idx=padding_idx)
    recurrent_layer = nn.RNN(1, 1, batch_first=True, nonlinearity="relu")
    recurrent_layer.bias_ih_l0[0] = 0.0
    recurrent_layer.bias_hh_l0[0] = 1.0
    recurrent_layer.weight_ih_l0[0, 0] = 1.0
    recurrent_layer.weight_hh_l0[0, 0] = 1.0

    # Get the word embeddings for the padded sequence
    word_embeddings = embedding_layer(padded)

    # With packing: pack the word embeddings, 
    # run the recurrent layer on the packed padded embeddings, 
    # and then unpack the results
    packed_word_embeddings = pack_padded_sequence(word_embeddings, [3, 4, 5], batch_first=True, enforce_sorted=False)
    (final_layer_all_timesteps, all_layers_final_timestep) = recurrent_layer(packed_word_embeddings)
    final_layer_all_timesteps = pad_packed_sequence(final_layer_all_timesteps, batch_first=True, padding_value=padding_idx)

    print("===========================")
    print("WITH PACKING")
    print("---------------------------")
    print("Recurrent layer activation at all timesteps, for each sequence:")
    print(final_layer_all_timesteps[0].view(3, -1))
    print("\nWhat PyTorch gets as the recurrent layer activation at the \"final\" timestep for each sequence:")
    for (seqnum, value) in enumerate(list(all_layers_final_timestep.view(-1))):
        print("Sequence {}: {}".format(seqnum + 1, int(value)))

    # Without packing: run the recurrent layer on the padded embeddings directly
    (final_layer_all_timesteps, all_layers_final_timestep) = recurrent_layer(word_embeddings)

    print("\n===========================")
    print("WITHOUT PACKING")
    print("---------------------------")
    print("Recurrent layer activation at all timesteps, for each sequence:")
    print(final_layer_all_timesteps.view(3, -1))
    print("\nWhat PyTorch gets as the recurrent layer activation at the \"final\" timestep for each sequence:")
    for (seqnum, value) in enumerate(list(all_layers_final_timestep.view(-1))):
        print("Sequence {}: {}".format(seqnum + 1, int(value)))

**QUESTION 1.1.2. [2 points]**  
Which timestep in a sequence is treated as "final" with packing, and which timestep is treated as "final" without packing? What has to be true about a sequence in order for these two definitions of the "final" timestep to be the same? In sequences where they are *not* the same, how/why does using padding without packing make the value of the recurrent layer activation at the "final" timestep misleading?

*Double-click on this cell to enter your response.*

---
---

Now that we have padded our data, we can use it to train the RNN. Training looks exactly the same as it does for other NN models we have seen before:  

1. set the seed, in order to ensure results are reproducible;  
2. create the model as an instance of the relevant class;  
3. define an optimizer for the model, which will be used to nudge the weights during training;  
4. train the model for a specified number of epochs.

Since calculating the gradients in RNNs involves many steps, the `SGD` optimizer that we used in the past is typically too simple to enable good performance. We will still use it here, but we will explore alternatives later in this notebook.

---
---
### Questions 1.1.3 - 1.1.10 

The code cell below trains a RNN classifier for sentiment analysis on the movie reviews dataset we used in Assignment 1, for which the number of units in the recurrent layer has been set to the size of an embedding (third argument `embeddings.dim`). Run the code cell to train the model, and then answer the following questions.

*Note: it may take a few minutes to train the model*

In [ ]:
torch.manual_seed(1)
rnn_classifier = RNNClassifier(vocab, embeddings, embeddings.dim, 20)
optimizer = optim.SGD(rnn_classifier.parameters(), lr=0.01)
train(rnn_classifier, train_dataloader_rnn, optimizer, epochs=100, print_every=5, validation_data=test_dataloader_rnn)

**QUESTION 1.1.3. [2 points]**  
Focus first on the training accuracy and training loss, presented in the second and third columns of the table above. How do these numbers change as training epochs pass? What does that mean?

*Double-click on this cell to enter your response.*

**QUESTION 1.1.4. [2 points]**  
Now focus on the validation accuracy and validation loss, presented in the fourth and fifth columns. How do these numbers change as training epochs pass? How does that compare to what happens with the training accuracy and training loss? What does that suggest?

*Double-click on this cell to enter your response.*

---

To put the RNN model in context, it is helpful to compare it against a bag-of-embeddings classifier like the one we saw in Assignment 1. The following code cell creates and trains such a model. Run it, then answer the following questions.

In [ ]:
torch.manual_seed(1)
boe_classifier = BagOfEmbeddingsBinaryClassifier(vocab, embeddings, 20)
optimizer = optim.SGD(boe_classifier.parameters(), lr=0.01)
train(boe_classifier, train_dataloader_boe, optimizer, epochs=100, print_every=5, validation_data=test_dataloader_boe)

**QUESTION 1.1.5. [2 points]**  
How does performance on the training set differ between the RNN and BoE models? What does that mean for the potential to capture complex patterns in the RNN model as compared to the BoE model, and where does that potential come from?  

*Double-click on this cell to enter your response.*

**QUESTION 1.1.6. [2 points]**  
How does performance on the validation set differ between the RNN and BoE models? What does that mean for the risk of overfitting in the RNN model as compared to the BoE model, and where does that risk come from?  

*Double-click on this cell to enter your response.*

**QUESTION 1.1.7. [1 point]**  
How does training time differ between the RNN and BoE models, and why?  

*Double-click on this cell to enter your response.*

---

So far, it might not be entirely clear why we might choose an RNN over a BoE model. To elucidate this point, it can be helpful to compare predictions of the two models on the same constructed data.

**QUESTION 1.1.8. [1 point]**  
The code cell below generates predictions from the RNN model for two texts that swap the positions of two words, and the code cell after that generates predictions from the BoE model for the same texts. Run the code cells to see the predicted classifications. Why do the two texts get the same prediction under the BoE model but different predictions under the RNN model? Which model seems to be best?

In [ ]:
print("\nRNN predictions:")
predict_multiple(rnn_classifier, ["I loved it at first, but by the end I hated it", "I hated it at first, but by the end I loved it"], collate_batch_rnn)

In [ ]:
print("\nBoE predictions:")
predict_multiple(boe_classifier, ["I hated it at first, but by the end I loved it", "I loved it at first, but by the end I hated it"], collate_batch_boe)

*Double-click on this cell to enter your response.*

---

Exploring the effects of changing the texts on the RNN's predictions can also help us understand what it might be doing.

**QUESTION 1.1.9. [2 points]**  
The code cell below generates predictions from the RNN model for two texts that differ minimally from the second text you examined above, by adding an intensifier ("really") at different positions. Does adding the intensifier early or late affect the classification probability more? Does this agree with your expectations about the effect of the intensifier on sentiment? Suggest a technical reason why the model might show this difference.

In [ ]:
predict_multiple(rnn_classifier, ["I really loved it at first, but by the end I hated it", "I loved it at first, but by the end I really hated it"], collate_batch_rnn)

*Double-click on this cell to enter your response.*

**QUESTION 1.1.10. [2 points]**  
The code cell below generates a prediction from the RNN model for a text that is similar to the ones you examined above, but with a different ordering of information. Does the prediction agree with your expectations, and with the models predictions for previous sentences? Suggest a reason why or why not.

In [ ]:
predict_multiple(rnn_classifier, ["I hated it in the end, even though I loved it at first"], collate_batch_rnn)

*Double-click on this cell to enter your response.*

---
---

## Activity 1.2: Learning about words in RNNs

We saw in Assignment 1 that we could improve the performance of a bag-of-embeddings classifier by allowing it to continue to learn about the meanings of words, by *fine-tuning* the embeddings. We can allow the embeddings to be fine-tuned in an RNN in the same way, by setting `freeze=False` when we load the embeddings inside the model. The `RNNClassifier` class we have provided has an argument `freeze_embeddings` that controls this setting. In the model we trained before, `freeze_embeddings` was set to `True` (the default value), which means that the embeddings were not fine-tuned; let's see what happens when we set it to to `False`.

In [ ]:
torch.manual_seed(1)
unfrozen_rnn_classifier = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.SGD(unfrozen_rnn_classifier.parameters(), lr=0.01)
train(unfrozen_rnn_classifier, train_dataloader_rnn, optimizer, epochs=100, print_every=5, validation_data=test_dataloader_rnn)

---
---
### Questions 1.2.1 - 1.2.2

For these questions, you should compare the results of training the RNN model that fine-tunes embeddings (above) with the model that leaves embeddings frozen (which you trained for [Question 1.1.3](#Questions-1.1.3---1.1.10)).

**QUESTION 1.2.1. [2 points]**  
Look at the training and validation accuracies and losses in the last few epochs of training, and at the training time for the model. In what ways (if any) does it seem like allowing the model to fine-tune the embeddings has helped its performance? In what ways (if any) does it seem like fine-tuning has hurt performance?  

*Double-click on this cell to enter your response.*

**QUESTION 1.2.2. [2 points]**  
At which epochs are the gains in performance most noticeable? Would you say that fine-tuning the embeddings gave a rapid improvement (i.e., even from the first few epochs) or a slow improvement (i.e., it took a while for it to really show up)? Suggest a reason why that might be the case.

*Double-click on this cell to enter your response.*

---
---

One way to examine the influence of allowing embeddings to be fine-tuned is to see how specific embeddings change over the course of training. For the next part of the activity, we will add new tokens (`"<newneg>"` and `"<newpos>"`) to all negative and positive (respectively) training and validation samples. These new tokens will start with embeddings composed entirely of zeros, meaning that they are assessed to have "no meaning" by the model. We will explore how these embeddings change after a few epochs of training, as the model learns that `"<newneg>"` is distinctive of negative reviews and `"<newpos>"` is distinctive of positive reviews.

In particular, we will create *three versions* of the data, based on inserting these new tokens at different positions: one version which always inserts it at the *start* of a text sequence; one version which always inserts it in the *middle* of a text sequence; and one version which always inserts it at the *end* of a text sequence. We will compare the rates at which the model learns about the new tokens in these different positions.

Run the code cell below to create the new versions of the data, and to create a key function that we'll use to explore the results.

In [ ]:
if "<newneg>" not in vocab:
    vocab.append_token("<newneg>")
    vocab.append_token("<newpos>")

def collate_batch_rnn_newwords(batch, add_at="start"):
    """Converts a batch of sequence data into padded and packed PyTorch 
    tensor format, and collates the results by label, text, and sequence
    length, for use in a RNN model.
    Adds a new token by class (<newpos> or <newneg>) at the start, middle,
    or end of each text, as indicated by add_at
    """
    # Initialize lists that separate out the two components
    label_list = list()
    text_list = list()
    seq_lengths = list()
    
    for (label, text) in batch:
        # Get index of word to add
        new_word = "<new{}>".format(label)
        new_index = torch.tensor(vocab([new_word]), dtype=torch.int64)
        # Convert to PyTorch format
        (label_index, text_indices) = data_to_indices((label, text))
        # Add the new word index at the relevant spot
        if add_at == "start":
            left_part = torch.tensor([], dtype=torch.int64)
            right_part = text_indices
        elif add_at == "end":
            left_part = text_indices
            right_part = torch.tensor([], dtype=torch.int64)
        elif add_at == "middle":
            middle_index = len(text_indices) // 2
            left_part = text_indices[:middle_index]
            right_part = text_indices[middle_index:]
        text_indices = torch.cat([left_part, new_index, right_part])
        # Add converted data to separate component lists
        label_list.append(label_index)
        text_list.append(text_indices)
        seq_lengths.append(len(text_indices))
    
    # Convert to mini-batch tensors
    label_tensor = torch.tensor(label_list, dtype=torch.int64)
    text_tensor = pad_sequence(text_list, batch_first=True, padding_value=padding_idx)
    
    return (label_tensor, text_tensor, seq_lengths)
          
train_dataloader_newstart = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "start"))
train_dataloader_newmiddle = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "middle"))
train_dataloader_newend = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "end"))

test_dataloader_newstart = DataLoader(test_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "start"))
test_dataloader_newmiddle = DataLoader(test_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "middle"))
test_dataloader_newend = DataLoader(test_data, batch_size=8, shuffle=True, collate_fn=lambda batch: collate_batch_rnn_newwords(batch, "end"))

def newword_similarities(model):
    with torch.no_grad():
        word_embeddings = model.get_parameter("embedding.weight")
        embedding_similarities = pd.DataFrame({
            "word1": ["<newneg>", "<newneg>", "<newneg>", "<newpos>", "<newpos>"],
            "word2": ["<newpos>", "boring", "outstanding", "boring", "outstanding"]
        })
        embedding_similarities["similarity"] = (
            embedding_similarities.apply(lambda row: 
                                         float(
                                             torch.nn.functional.cosine_similarity(
                                                 word_embeddings[vocab([row["word1"]])[0]],
                                                 word_embeddings[vocab([row["word2"]])[0]], 
                                                 dim=0)
                                         ), axis=1))
    return embedding_similarities

For comparison, here is what training looks like when a new token is added to the start of each sequence, but we do not allow the embeddings to be fine-tuned. You can think of this like a model that *doesn't* add any new tokens to the sequence, but rather initializes the recurrent layer to the same non-zero value for each sequence (whereas the regular RNN initializes it to zero).

In [ ]:
torch.manual_seed(1)
rnn_classifier_newstart_baseline = RNNClassifier(vocab, embeddings, embeddings.dim, 20)
optimizer = optim.SGD(rnn_classifier_newstart_baseline.parameters(), lr=0.01)
train(rnn_classifier_newstart_baseline, train_dataloader_newstart, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newstart)

---
---
### Questions 1.2.3 - 1.2.11



We'll start by looking at what happens when we put the new tokens at the *start* of each text sequence and allow fine-tuning of embeddings.

Run the code cell below to train the model.

In [ ]:
torch.manual_seed(1)
rnn_classifier_newstart = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.SGD(rnn_classifier_newstart.parameters(), lr=0.01)
train(rnn_classifier_newstart, train_dataloader_newstart, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newstart)

**QUESTION 1.2.3. [2 points]**  
Compare the training performance to what you saw just above, when the same model *did not* allow fine-tuning of embeddings. Have the training and validation accuracies and losses changed much? Why / why not? How about the training time? Why / why not?

*Double-click on this cell to enter your response.*

Run the following code cell to generate a report of the cosine-similarity between the fine-tuned `"<newneg>"` and `"<newpos>"` embeddings, as well as the cosine-similarity of each with two words that are distinctive of negative and positive reviews (`"boring"` and `"outstanding"`, respectively).

In [ ]:
newword_similarities(rnn_classifier_newstart)

**QUESTION 1.2.4. [2 points]**  
Before training, each of these cosine-similarities were 0, because the new embeddings were initialized to contain all zeros. What do you notice about the cosine-similarity of `"<newneg>"` and `"<newpos>"` now: is it positive or negative, and how big is it? What does that mean?

*Double-click on this cell to enter your response.*

**QUESTION 1.2.5. [2 points]**  
What about the cosine-similarities between each new token and the two reference words? Do they make sense? What does that suggest about what the model has learned about these new tokens?

*Double-click on this cell to enter your response.*

---
Now, for comparison, let's look at what happens when we put the new tokens at the *end* of each sequence and allow them to be fine-tuned.

Run the code cell below to train the model.

In [ ]:
torch.manual_seed(1)
rnn_classifier_newend = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.SGD(rnn_classifier_newend.parameters(), lr=0.01)
train(rnn_classifier_newend, train_dataloader_newend, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newend)

**QUESTION 1.2.6. [1 point]**  
How much have the training and validation accuracies and losses changed now? Why is that? Suggest a reason why this is so different to the previous case.  

*Double-click on this cell to enter your response.*

It also helps to examine the embeddings for this model. Run the code cell below to generate the report.

In [ ]:
newword_similarities(rnn_classifier_newend)

**QUESTION 1.2.7. [1 point]**  
What do you notice about the cosine-similarity of `"<newneg>"` and `"<newpos>"` now, and what does that mean? Suggest a reason why it is so different to what you saw in the previous case.  

*Double-click on this cell to enter your response.*

**QUESTION 1.2.8. [1 point]**  
What about the cosine-similarities between each new token and the two reference words? Do they make sense now? What does that suggest about what the model has learned about these new tokens?

*Double-click on this cell to enter your response.*

---
One of the things that we have to be very aware of in RNNs - especially when fine-tuning embeddings - is *gradients*. We have to be on the lookout for *vanishing gradients* and *exploding gradients*.

The code cell below prints the gradients of the embeddings for the new tokens when they are inserted at the start, middle, and end of each sequence. Run the code cell to see the gradients.

(Note that gradients come from considering the *loss*, which can only be measured over a set of inputs. We have calculated these gradients over the inputs in the validation set.)

In [ ]:
def view_untrained_gradients(word_position, model_class, model_args, model_kwargs):
    torch.manual_seed(1)
    untrained_classifier = model_class(*model_args, **model_kwargs)

    # Provide the whole validation set to get gradients
    for (label_list, *datapoint_list) in eval("test_dataloader_new{}".format(word_position)):
        # Run the forward pass to make predictions for the mini-batch
        predicted_probs = untrained_classifier(*datapoint_list).view(-1)
        # Compute the loss and send it backward through the network to get gradients
        # Note: PyTorch averages the loss over all datapoints in the minibatch
        loss = untrained_classifier.loss_function(predicted_probs, label_list.to(torch.float32))
        loss.backward()

    # Now look at the gradients
    word_embeddings = untrained_classifier.get_parameter("embedding.weight")
    (newneg_grad, newpos_grad) = word_embeddings.grad[-2:]
    print("Gradient for <newneg> inserted at {}:\n{}".format(word_position, newneg_grad))
    print("\nGradient for <newpos> inserted at {}:\n{}".format(word_position, newpos_grad))

for word_position in ["start", "middle", "end"]:
    view_untrained_gradients(word_position, RNNClassifier, 
                             (vocab, embeddings, embeddings.dim, 20),
                             {"freeze_embeddings": False})
    print("-----------------------\n")

**QUESTION 1.2.9 [2 points]**  
What do you notice about how the gradients change between versions, both in terms of the size of the gradients and the correspondences between the positive and negative gradients for each version? Why do you think that is?

*Double-click on this cell to enter your response.*

---
It's clear from the above exploration that inserting the new tokens at the beginning and end results in drastically different gradients. The gradients determine how quickly the model can learn the meanings of the new tokens. In turn, this affects how quickly the model can improve its performance; each sequence always contains one of these tokens that identifies its sentiment, so the faster the model can learn their meaning, the more it can leverage that to make sure that it interprets the sentiment of the sequence correctly.

However, learning about these tokens is not guaranteed to drastically improve performance! The model also has to be able to *retain* the information that it gets when it sees those tokens, until the point where it does the classification task. This is well demonstrated by the version of the model where the new tokens are always placed in the middle of the sequence:

In [ ]:
torch.manual_seed(1)
rnn_classifier_newmiddle = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.SGD(rnn_classifier_newmiddle.parameters(), lr=0.01)
train(rnn_classifier_newmiddle, train_dataloader_newmiddle, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newmiddle)

newword_similarities(rnn_classifier_newmiddle)

**QUESTION 1.2.10. [2 points]**  
Based on the gradients you saw earlier, would you expect the embeddings for the new tokens in this model to be better (more meaningful) or worse (less meaningful) than in the model where the new tokens are always placed at the start of the sequence? The cosine-similarity report for this model is a little mixed: identify one way in which it suggests that the embeddings are better, and one way in which it suggests that the embeddings are worse. Why do you think it is so mixed?

*Double-click on this cell to enter your response.*

**QUESTION 1.2.11. [2 points]**  
Let's assume that the embeddings for the new tokens are, on balance, better in this model than in the model where the new tokens are added to the start of a sequence. Under this assumption, we would expect the performance of this model (in terms of training and validation accuracies and losses) to be better as well. Is this the case? Suggest a reason why or why not (other than the assumption not being justified).

*Double-click on this cell to enter your response.*

---
---

## Activity 1.3: Improving the RNN model

In Activity 1.2, we saw two main issues that can limit performance of RNNs: *gradients* and *retention*. In this activity, we'll explore some changes that improve the RNN model by addressing these issues.

We'll start with a change that addresses retention without directly affecting gradients: changing the size of the recurrent (hidden) layer.

One of the big issues for retention is the fact that all of the information the network keeps track of in its recurrent (hidden) layer has to be compressed to a relatively small number of units. In the models we have been using so far, we've had 50-unit word embeddings and 50-unit recurrent layers; so the model has been forced to compress a whole document down to the size of a single word! While there are certainly some redundancies that make this kind of compression reasonable - for example, the fact that words have many aspects of meaning expressed in their embeddings, and only a few of these may be relevant for the classification task at hand - it can still be beneficial to take a little pressure off by making the recurrent layer larger than the embedding layer.

---
---
### Questions 1.3.1 - 1.3.2

We saw an issue of retention with the model where the new tokens were inserted in the middle of the sequence. The code cell below trains a new version of this model, with 100 units in the recurrent layer rather than 50. Run the code cell to train the model and present the cosine-similarity report.

In [ ]:
torch.manual_seed(1)
rnn_classifier_newmiddle_bigger = RNNClassifier(vocab, embeddings, 100, 20, freeze_embeddings=False)
optimizer = optim.SGD(rnn_classifier_newmiddle_bigger.parameters(), lr=0.01)
train(rnn_classifier_newmiddle_bigger, train_dataloader_newmiddle, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newmiddle)

newword_similarities(rnn_classifier_newmiddle_bigger)

**QUESTION 1.3.1. [2 points]**  
Compared to the corresponding model you trained for Question 1.2.10, how are the training and validation accuracies and losses and the training time different here? Why does increasing the size of the recurrent layer have these effects?

*Double-click on this cell to enter your response.*

**QUESTION 1.3.2. [1 point]**  
Does it seem like the quality of the embeddings the model has learned for the new tokens has changed in any way? Why do you think this is, if the gradients haven't changed?

*Double-click on this cell to enter your response.*

---
---

Next, we'll consider a change that addresses how the gradients are used in training, without directly affecting retention: changing the optimizer.

Up to now, we have been using the `SGD` optimizer, which nudges the weights by subtracting an amount proportional to the gradient, where the proportionality scaling factor is given by the *learning rate*. In `SGD`, the learning rate is the same for all weights across all epochs, and the nudge is solely determined by the learning rate and the current gradient. For RNNs, it is practically always better to use the `Adam` optimizer, which allows the learning rate to *decay* over epochs and to be *different* for weights that are nudged often vs. weights that are nudged rarely. It also introduces *momentum*, which lets the weights continue to be nudged according to a "memory" of what the gradient used to be like (rather than just the current gradient). 

---
---
### Question 1.3.3

Using the `Adam` optimizer in PyTorch is easy: just swap `optim.SGD()` for `optim.Adam`, and remove the `lr` keyword argument (since Adam figures out the learning rate itself). The code cell below illustrates, for the model where the new tokens are added at the start of the sequence.

In [ ]:
torch.manual_seed(1)
rnn_classifier_newstart_adam = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.Adam(rnn_classifier_newstart_adam.parameters())
train(rnn_classifier_newstart_adam, train_dataloader_newstart, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newstart)

newword_similarities(rnn_classifier_newstart_adam)

**QUESTION 1.3.3. [2 points]**  
Compared to the corresponding model you trained for Question 1.2.3, how are the training and validation accuracies and losses different here? What about the cosine-similarity report on the quality of the embeddings learned for the new tokens (comared to Question 1.2.4). What does this mean?

*Double-click on this cell to enter your response.*

---
---

Now let's consider a way of changing both retention and gradients: making the RNN bidirectional. In a bidirectional RNN, the model is trained both forward and backward in time on each sequence, and the document embedding is obtained by concatenating the final forward recurrent layer and the final backward recurrent layer. This means that:  

1. the final document embedding is twice as large (like in Questions 1.3.1 - 1.3.2); and  
2. tokens at both the start and the end of the sequence are encountered "right before" the final document embedding is created.

PyTorch makes training a bidirectional RNN easy, through a Boolean argument `bidirectional` in `nn.RNN()`; in our `RNNClassifier()` class, this is provided through the `recurrent_bidirectional` keyword argument. The `forward()` method also needs to be updated to reflect the fact that the document embedding is twice the size of `recurrent_dim` (our `RNNClassifier()` class takes care of this automatically).

---
---
### Questions 1.3.4 - 1.3.6

Run the code cell below to train a bidirectional RNN on the data where new tokens have been added at the start of each sequence. (This model still uses the `SGD` optimizer, for comparability with earlier models; when `Adam` is used instead, it hits a performance ceiling within 3 epochs).

In [ ]:
torch.manual_seed(1)
birnn_classifier_newstart = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False, recurrent_bidirectional=True)
optimizer = optim.SGD(birnn_classifier_newstart.parameters(), lr=0.01)
train(birnn_classifier_newstart, train_dataloader_newstart, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newstart)

newword_similarities(birnn_classifier_newstart)

**QUESTION 1.3.4. [2 points]**  
What do you notice about the training performance, training time, and cosine-similarity report, as compared to the corresponding unidirectional model you trained for Question 1.2.3? Why do these differences exist?

*Double-click on this cell to enter your response.*

The code cell below prints the gradients for the bidirectional RNN based on the position at which the new tokens are inserted. 

In [ ]:
for word_position in ["start", "middle", "end"]:
    view_untrained_gradients(word_position, RNNClassifier, 
                             (vocab, embeddings, embeddings.dim, 20),
                             {"freeze_embeddings": False, "recurrent_bidirectional": True})
    print("-----------------------\n")

**QUESTION 1.3.5. [2 points]**  
In broad strokes, what is the same as you saw for the gradients before (for Question 1.2.9), and what is different? Why?

*Double-click on this cell to enter your response.*

The code cell below trains a bidirectional RNN on the data where the new tokens are inserted in the middle of a sequence.

In [ ]:
torch.manual_seed(1)
birnn_classifier_newmiddle = RNNClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False, recurrent_bidirectional=True)
optimizer = optim.SGD(birnn_classifier_newmiddle.parameters(), lr=0.01)
train(birnn_classifier_newmiddle, train_dataloader_newmiddle, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newmiddle)

newword_similarities(birnn_classifier_newmiddle)

**QUESTION 1.3.6. [1 point]**  
Why is this model not as good as the one where the new tokens are inserted at the start of the sequence?  

*Double-click on this cell to enter your response.*

---
---

To help the model deal with the case where the new tokens are inserted in the middle of the sequence, we need to make one final change that affects both retention and gradients: changing from a simple RNN architecture to an LSTM architecture. 

Again, PyTorch makes this easy: just substitute the `nn.RNN()` layer in the model for a lookalike `nn.LSTM()` layer. The code cell below creates a `LSTMClassifer` class that is a factory for making LSTM models:

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self, vocab, embeddings, recurrent_dim, hidden_dim, freeze_embeddings=True,
                 recurrent_layers=1, recurrent_bidirectional=False):
        super(LSTMClassifier, self).__init__()
        
        self.vocab = vocab
        
        vocab_embeddings = embeddings.get_vecs_by_tokens(self.vocab.get_itos())
        padding_idx = self.vocab.get_stoi().get("<pad>")  # Get the <pad> index
        self.embedding = nn.Embedding.from_pretrained(vocab_embeddings, freeze=freeze_embeddings, 
                                                      padding_idx=padding_idx) # Tell PyTorch that <pad> is for padding
        
        # The embeddings go into an RNN layer with recurrent_dim units
        self.recurrent_layer = nn.LSTM(embeddings.dim, recurrent_dim,
                                       num_layers=recurrent_layers, bidirectional=recurrent_bidirectional,
                                       batch_first=True) # Because we'll make the mini-batch a list of sequences
        
        # The recurrent output creates a doc_embedding, which feeds into a of hidden_dim units
        # We'll be concatenating the forward and backward direction of all layers
        # from the recurrent output, so the doc_embedding will be sized accordingly
        doc_embedding_dim = recurrent_dim * recurrent_layers * int(1 + recurrent_bidirectional)
        self.hidden_layer = nn.Sequential(
            nn.Linear(doc_embedding_dim, hidden_dim),
            nn.ReLU()
        )
        
        # The output layer will go from the hidden layer (hidden_dim units) to a single unit
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
        self.loss_function = nn.BCELoss()

    def forward(self, padded_text, seq_lengths):
        word_embeddings = self.embedding(padded_text)
        
        # The sequence of word embeddings has to be packed for efficiency of the RNN
        packed_word_embeddings = pack_padded_sequence(word_embeddings, seq_lengths, batch_first=True, enforce_sorted=False)
        (final_layer_all_timesteps, (all_layers_final_timestep, _)) = self.recurrent_layer(packed_word_embeddings)
        
        # all_layers_final_timestep contains the activations of all (stacked / bidirectional) recurrent
        # layers at the final timestep for each sequence (taking the padding into account).
        # For our classifier, we will stick all of these layers together (forward + backward, 
        # for each stacked layer) to use as the document embedding.
        # all_layers_final_timestep has shape (num_layers, minibatch_size, recurrent_dim);
        # we want something of shape (minibatch_size, num_layers * recurrent_dim),
        # so we reorder the dimensions and then reshape to stick everything together
        minibatch_size = all_layers_final_timestep.size(1)
        doc_embedding = all_layers_final_timestep.permute(1, 0, 2).reshape(minibatch_size, -1)
        
        hidden = self.hidden_layer(doc_embedding)
        output = self.output_layer(hidden)
        return output

---
---
### Questions 1.3.7 - 1.3.8

Let's train a unidirectional LSTM on the data where new tokens have been added at the start of the sequence (the hardest case!), using the `Adam` optimizer.

In [ ]:
torch.manual_seed(1)
lstm_classifier_newstart = LSTMClassifier(vocab, embeddings, embeddings.dim, 20, freeze_embeddings=False)
optimizer = optim.Adam(lstm_classifier_newstart.parameters())
train(lstm_classifier_newstart, train_dataloader_newstart, optimizer, epochs=10, print_every=1, validation_data=test_dataloader_newstart)

newword_similarities(lstm_classifier_newstart)

**QUESTION 1.3.7. [2 points]**  
Compare this model to the corresponding RNN you trained for question 1.3.3. What does this model seem to have done better? Intuitively, what is it about the LSTM architecture that you think has let it make those improvements?

*Double-click on this cell to enter your response.*

**QUESTION 1.3.8. [2 points]**  
Is there anything that this model seems to do worse, or that we might want to be wary of when using LSTMs in general? Intuitively, what is it about the LSTM architecture that is behind these (potential) issues?

*Double-click on this cell to enter your response.*

---
---

# Part 2: Simple RNNs from scratch [50 pts]

In this part of the assignment, you'll build an understanding of what's going on "under the hood" in simple recurrent networks, by implementing the forward and backward passes of simple RNNs yourself, and using them to train a network for a single epoch. (We won't put everything together into a class like we did in Assignment 1, but you could do so if you wanted to!)

Run the code cell below first, to import the modules you'll need for this part of the assignment. The code cell also defines several functions that you wrote in Assignment 1. *(Note: the function that was previously `logistic_derivative()` is now `sigmoid_derivative()` and the function that was previously `bce_loss_gradient()` is now `crossent_loss_gradient()`. We have also added (optional) keyword arguments to `calculate_weights_gradient()` and `send_delta_back()` to account for the fact that the recurrent weights in RNNs do not have bias terms.)*

*Note: this assignment will use NumPy, but you will not need to know any more about it than what you needed for Assignment 1.*

In [ ]:
import numpy as np
import copy
from util import *
    
def insert_bias(values):
    """Inserts a 1 at the beginning of a vector of values."""
    return np.insert(values, 0, 1)

def logistic_layer(values, weights):
    """Calculates the result of passing a set of values through a logistic
    layer of a neural network with given weights.
    
    Arguments
    ---------
    values: np.array representing a vector of size (num_features,);
            does not contain a value for the bias term
    weights: np.array representing a vector of size (num_features + 1,)
             or a matrix of size (num_outputs, num_features + 1);
             the first element (in each row) corresponds to the bias term
    
    Returns
    -------
    activations: if weights is a vector, this is a float;
                 if weights is a matrix, this is a vector of size (num_outputs,)
    """
    values = insert_bias(values)
    evidence = linear(weights, values)
    activations = sigmoid(evidence)
    return activations

def crossent_loss_gradient(predicted, actual):
    """Returns the gradient of the cross-entropy loss for a single training sample,
    based on the predicted probability and the actual class for that sample
    
    Arguments
    ---------
    predicted: np.array or float; the probability(ies) output by the classifier
               for a given training sample
    actual: np.array or int; the actual class / probability(ies) for the training
            sample (0 for default, 1 for alternative)
    """
    return predicted - actual

def calculate_weights_gradient(delta, values, insert_bias_term=True):
    """Combines local gradients delta with feature values in order to 
    calculate the weights gradient of a layer of a neural network.
    
    Arguments
    ---------
    delta: np.array representing a vector of size (num_units,),
           containing the local gradient of each unit with respect to its input
    values: np.array representing a vector of size (num_features,),
            containing the values of features that combine with weights to
            form inputs to the units; does not contain a value for the bias term
    insert_bias_term: bool (default True); indicates whether to insert an element to the
                      values vector to account for the bias term (i.e., whether the weights
                      matrix being nudged has a row for the bias term)
    
    Returns
    -------
    weights_gradient: np.array representing a matrix of size (num_units, num_features + 1)
                      (this is a row vector if num_units == 1), which contains the gradient
                      of the loss function with respect to the weights of the current layer
    """
    if insert_bias_term:
        values = insert_bias(values)
    weights_gradient = np.outer(delta, values)
    if isinstance(weights_gradient, np.ndarray) and weights_gradient.shape[0] == 1:
        weights_gradient = weights_gradient.flatten()
    return weights_gradient

def send_delta_back(delta, weights, remove_bias_term=True):
    """Obtains a signal vector by sending local gradients backward through
    weights. The element of the signal vector corresponding to the bias unit
    can be removed prior to returning, so that the output has the same shape as
    the previous layer activation derivatives.
    
    Arguments
    ---------
    delta: np.array representing a vector of size (curr_layer_size,),
           containing the local gradients at the current layer (i)
    weights: np.array representing a matrix of size (prev_layer_size + 1, curr_layer_size),
             containing the weights that lead into the current layer (i)
    remove_bias_term: bool (default True); indicates whether to remove the element of the signal
                      vector prior to returning (i.e., whether the weights matrix has a row for
                      the bias term)
             
    Returns
    -------
    signal: np.array representing a vector of size (prev_layer_size,),
            containing the signal (precursor to local gradients) at the
            previous layer (i-1). The element corresponding to the bias
            unit has been removed.
    """
    signal = np.atleast_2d(weights).T @ np.atleast_1d(delta) # make sure the shapes align
    signal = signal.flatten() # convert back to a vector
    if remove_bias_term:
        trimmed_signal = np.delete(signal, 0) # remove the first element
        return trimmed_signal
    else:
        return signal

def linear(weights, values):
    """Returns the matrix algebra product of weights and values,
    both represented as NumPy arrays.
    """
    return weights @ values

def sigmoid(evidence):
    """Applies the logistic function to convert evidence to probability."""
    return 1 / (1 + np.exp(-evidence))

def sigmoid_derivative(logistic_output):
    """Returns the derivative of the logistic function, calculated from the
    output of the logistic function at a particular point.
    """
    return logistic_output * (1 - logistic_output)

def tanh(z):
    """Applies the tanh function to calculate activation, given input z."""
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

def tanh_derivative(tanh_output):
    """Returns the derivative of the tanh function, calculated from the
    output of the tanh function at a particular point.
    """
    return 1 - (tanh_output ** 2)

def relu(z):
    """Applies the ReLU function to calculate activation, given input z."""
    return np.maximum(z, 0)

def relu_derivative(relu_output):
    """Returns the derivative of the ReLU function, calculated from the
    output of the ReLU function at a particular point.
    """
    if isinstance(relu_output, np.ndarray):
        return (relu_output > 0).astype(int)
    else:
        return int(relu_output > 0)

## Activity 2.1: The forward pass in a sequence classifier

In this first activity, you'll implement the forward pass of a unidirectional (left-to-right) recurrent neural classifier with a single hidden layer (which uses ReLU by default, but which can be changed easily).

Here is a visualization of the structure of the network:

![RNN sequence classifier structure](https://github.com/ucsb-ling111/assignment2-giovani-gutierrez/blob/main/images/rnn_classifier.png?raw=1)

*Note: in this activity, we will assume that the sequence embedding, which is the activation of the hidden layer at the final timestep, is used directly as the input to the output layer. This is slightly different to the example we saw in class, where there was a hidden layer between the sequence embedding and the output layer.*

We'll use a slight adaptation of the notation from the textbook in this activity:  
- $\mathbf{x}_t$ is the vector of values that is input to the network at time step $t$, which has shape `(num_features,)`;  
- $\mathbf{z}_t$ is the vector of inputs to the hidden layer activation function at time step $t$, which has shape `(num_hidden,)`;  
- $\mathbf{h}_t$ is the vector of hidden unit activations at time step $t$, which has shape `(num_hidden,)`;  
- $y$ is the input to the output unit after the entire sequence has been processed;  
- $p$ is the output of the network after the entire sequence has been processed, which represents the probability of deviating from the default class for that sequence;  
- $W$ is the matrix of *input weights* leading from the inputs of a given timestep, $\mathbf{x}_t$, to the hidden layer of that timestep, $\mathbf{h}_t$. This is a common matrix of shape `(num_hidden, num_features + 1)` that is used at all timesteps;  
- $U$ is the matrix of *recurrent weights* leading from the hidden layer of the previous timestep, $\mathbf{h}_{t-1}$, to the hidden layer of the current timestep, $\mathbf{h}_t$. This is a common matrix of shape `(num_hidden, num_hidden)` that is used between all pairs of adjacent timesteps;  
- $\mathbf{v}^T$ is the vector of *output weights* leading from the hidden layer of the final timestep, $\mathbf{h}_n$, to the output, $y$. It has shape `(num_hidden + 1,)` *(note: the textbook denotes this as a matrix rather than a vector for generality)*.
- $g(\cdot)$ is the activation function of the hidden layer.  
- $\sigma(\cdot)$ is the logistic function.

With this notation, the forward pass for a sequence with $n$ timesteps (from 1 to $n$) can be represented as follows:

#### RNN CLASSIFIER CALCULATION STEPS

1. calculate the input to the hidden layer activation function at timestep 1, based on the input values at timestep 1: $\mathbf{z}_1$ = $W\mathbf{x}_1$
2. calculate the hidden layer activations at timestep 1: $\mathbf{h}_1$ = $g(\mathbf{z}_1)$
3. for each timestep $t$ from 2 to $n$:  
   1. calculate the input to the hidden layer activation function at that timestep, based on the input values at that timestep and the hidden layer activations at the previous timestep: $\mathbf{z}_t$ = $W\mathbf{x}_t + U\mathbf{h}_{t-1}$
   2. calculate the hidden layer activations at that timestep: $\mathbf{h}_t$ = $g(\mathbf{z}_t)$
4. calculate the input to the output activation function for the network, based on the hidden layer activations at the final timestep: $y$ = $\mathbf{v}^T \mathbf{h}_n$
5. calculate the network's output activation: $p$ = $\sigma(y)$

*Note: the shapes in the definitions above do not quite match up because of the bias terms, which are built into $W$ and $\mathbf{v}^T$ but not $\mathbf{x}_t$ and $\mathbf{h}_t$. We will explain how the bias terms work as we progress through the activity.*

Most of these steps are exactly the same as in a feedforward neural network! There are three big differences:  
- in step 3A, the hidden layer at timestep $t$ has *two* inputs, one from the sequence value at that timestep ($\mathbf{x}_t$), and one from the hidden layer at the previous timestep ($\mathbf{h}_{t-1}$);  
- the for loop in step 3 means the calculation steps could be performed any number of times, depending on the length of the input sequence;  
- throughout, the *same* input and recurrent weights ($W$ and $U$) are used repeatedly for calculation steps at different timepoints, rather than having different weights for each step of the calculation.

### Task 2.1.1: Calculating inputs to hidden layer activation functions [2 points]

We have provided a function `linear()` which multiplies a vector of values by a matrix of weights. In a feedforward neural network, we could use this function to calculate the input to the activation function of any layer, as follows:  
1. augment the values vector by inserting a value for the bias term, using the `insert_bias()` function;  
2. multiply the augmented values vector by the weights, using the `linear()` function.

For the hidden layer in a recurrent neural network, this works for the first timestep ([RNN classifier calculation step 1](#RNN-CLASSIFIER-CALCULATION-STEPS)), but not for other timesteps ([RNN classifier calculation step 3A](#RNN-CLASSIFIER-CALCULATION-STEPS)). To make it work in general, we have to bring in the hidden layer activations from the previous timestep (if they exist):  
1. augment the values vector by inserting a value for the bias term, using the `insert_bias()` function;  
2. multiply the augmented values vector by the input weights $W$, using the `linear()` function;  
3. multiply the vector of hidden layer activations from the previous timestep by the recurrent weights $U$, using the `linear()` function;  
4. add together the vectors resulting from (2) and (3).

Note that **only the values vector is augmented for the bias term**; the hidden layer activations from the previous timestep (in step 3) are directly multiplied by the recurrent weights, without first having to be augmented for a bias term. This is because we only need to account for the bias **once**, which we do by incorporating it into the input weights $W$ as in feedforward neural networks.

Write a function `recurrent_linear()` which adapts the `linear()` function for recurrent neural networks. Your function should take as arguments:  
- a vector of `current_values` (of shape `(num_features,)`); and  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term and the subsequent elements correspond to the elements of `values`);  

as well as **optional keyword arguments** for:  
- a vector of `prev_hidden` activations (of shape `(num_hidden,)`); and  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`).

It should return the input to the hidden layer activation function, `current_z` ($\mathbf{z}_t$). 

If the optional keyword arguments are provided, your function should carry out steps 1-4 of the calculation above; if they are not provided (i.e., if `prev_hidden` is `None`), it should only carry out steps 1-2.

In [ ]:
def recurrent_linear(current_values, input_weights,
                    prev_hidden=None, recurrent_weights=None):
    """Returns the input to the hidden layer activation function at the current
    timestep, based on the input values to the network at the current timestep
    and the hidden layer activations from the previous timestep (if they exist).
    
    Arguments
    ---------
    current_values: np.array representing a vector of size (num_features,);
                    represents the input to the network at the current timestep.
                    Does not contain a value for the bias term.
    input_weights: np.array representing a matrix of size (num_hidden, num_features + 1);
                   represents the weights between the network input and hidden layer.
                   The first element in each row corresponds to the bias term.
    prev_hidden: np.array representing a vector of size (num_hidden,);
                 represents the hidden layer activations from the previous timestep.
                 Does not contain a value for the bias term. 
                 If None, not included in calculations.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between previous and current hidden layers.
                       Does not contain values for bias terms.
    
    Returns
    -------
    current_z: a vector of size (num_hidden,)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(recurrent_linear)

### Task 2.1.2: Getting hidden layer activations at a new timestep [2 points]

At each timestep of a recurrent neural network, there are hidden layer activations to calculate ([RNN classifier calculation steps 3A-3B](#RNN-CLASSIFIER-CALCULATION-STEPS)). This involves calculating the input to the hidden layer activation function at that timestep (using the `recurrent_linear()` function) and then passing it through the activation function, which in this case is ReLU (using the `relu()` function).

Write a function `get_hidden()` that calculates the hidden layer activations at a new timestep. Your function should take as arguments:  
- a vector of `prev_hidden` activations (of shape `(num_hidden,)`;  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `input_values` (of shape `(num_features,)`);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term); and  
- an `activation_function` (which is `relu` by default). 

It should return a vector `hidden` (of shape `(num_hidden,)`) which contains the hidden layer activations at the current timestep.

*Note: as demonstrated in this example, functions can be passed as arguments to other functions! Pass the name of the function as argument, without parentheses after it that would cause it to be evaluated; then it can be used with the corresponding argument name. In this example, passing `activation_function=relu` as an argument means that you can use `activation_function(z)` inside the definition of `get_hidden()` and get back the same thing as if you had written `relu(z)` instead. Because the activation function is an argument, you can reuse `get_hidden()` exactly as written with different activation functions, e.g. `activation_function=tanh` or `activation_function=sigmoid`, without having to recreate different versions of `get_hidden()` that are specific for each activation function.*

In [ ]:
def get_hidden(prev_hidden, recurrent_weights, input_values, input_weights,
               activation_function=relu):
    """Gets the hidden layer activations at the current timestep.
    
    Arguments
    ---------
    prev_hidden: np.array representing a vector of size (num_hidden,);
                 represents the hidden layer activations from the previous timestep.
                 Does not contain a value for the bias term. 
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    input_values: np.array representing a vector of size (num_features,);
                  represents the inputs to the network at the current timestep.
                  Does not contain a value for the bias term.
    input_weights: np.array representing a matrix of size (num_hidden, num_features + 1);
                   represents the weights between the network input and hidden layer.
                   The first element in each row corresponds to the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    
    Returns
    -------
    hidden: a vector of hidden layer activations at the current timestep, of shape (num_hidden,)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_hidden)

### Task 2.1.3: Getting the activations at all hidden layers [3 points]

The input to a recurrent neural network is not just one values vector, but a whole *sequence* of values vectors, one for each timepoint. There is a corresponding sequence of hidden layer activations. In order to make predictions with the network, we only need access to the hidden layer activations from the *final* timepoint (after processing the entire sequence); however, in order to train the network, we will need access to the hidden layer activations from *each* timepoint.

Write a function `get_hidden_sequence()` that processes a sequence of input vectors through a recurrent neural network and returns a corresponding sequence of hidden layer activation vectors. Your function should take as arguments:  
- a list of values vectors `values_list` (where the entries in the list are vectors of shape `(num_features,)`, arranged in order from timestep 1 to timestep $n$);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`); and  
- an `activation_function` (which is `relu` by default). 

It should return a list of hidden layer activation vectors `hidden_list`, containing $n$ vectors each of shape `(num_hidden,)`, arranged in a corresponding order to `values_list`.

*Note: your function will need to perform everything contained within [RNN classifier calculation steps 1-3](#RNN-CLASSIFIER-CALCULATION-STEPS). You should be able to do so just by putting together the functions you have already written.*

*Hint: don't forget to pass the `activation_function` argument to `update_hidden()`, as in `update_hidden(..., activation_function=activation_function)`.*

In [ ]:
def get_hidden_sequence(values_list, input_weights, recurrent_weights, 
                        activation_function=relu):
    """Processes a list of input values vectors and returns a list of hidden layer
    activations at all timepoints.
    
    Arguments
    ---------
    values_list: list of np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector
                 of input values for a particular timestep.
                 None of these vectors contain a value for the bias term.
    input_weights: np.array representing a matrix of size (num_hidden, num_features + 1);
                   represents the weights between the network input and hidden layer.
                   The first element in each row corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    
    Returns
    -------
    hidden_list: a list of vectors of hidden layer activations at each timestep, 
                 each of shape (num_hidden,)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_hidden_sequence)

### Task 2.1.4: The complete forward pass [2 points]

In the forward pass, the network calculates the hidden layer activations at each timestep ([RNN classifier calculation steps 1-3](#RNN-CLASSIFIER-CALCULATION-STEPS)), and then makes a prediction based on the hidden layer activations at the final timestep ([RNN classifier calculation steps 4-5](#RNN-CLASSIFIER-CALCULATION-STEPS)). This prediction is made using a `logistic_layer()`, exactly like logistic regression or a feedforward neural network.

In order to be able to train the network efficiently, the forward pass should return not only the final prediction probability, but also the hidden layer activations at each timestep. These will be used in backprop, just like we saw in Assignment 1 for feedfoward neural networks.

Write a function `recurrent_classifier_forward()` that calculates the output and intermediate hidden layer activations of a 2-layer recurrent neural network, as described in the [RNN classifier calculation steps](#RNN-CLASSIFIER-CALCULATION-STEPS). Your function should take as arguments:  
- a list of values vectors `values_list` (where the entries in the list are vectors of shape `(num_features,)`, arranged in order from timestep 1 to timestep $n$);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first element corresponds to a bias term); and  
- an `activation_function` (which is `relu` by default). 

It should return a **tuple** with two elements, where:  
- the first element is the list of hidden layer activations for each timestep; and  
- the second element is the ultimate output of the network (i.e., the classification probability for the sequence).

*Hint: the addition of a bias term for the output prediction is already handled by the `logistic_layer()` function.*

*Hint: don't forget to pass the `activation_function` argument to `get_hidden_sequence()`, as in `get_hidden_sequence(..., activation_function=activation_function)`.*

In [ ]:
def recurrent_classifier_forward(values_list, input_weights, recurrent_weights, output_weights, 
                                 activation_function=relu):
    """Sends an input sequence forward through a 2-layer recurrent neural network
    that aims to classify the entire sequence.
    
    Arguments
    ---------
    values_list: list of np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector
                 of input values for a particular timestep.
                 None of these vectors contain a value for the bias term.
    input_weights: np.array representing a matrix of size (num_hidden, num_features + 1);
                   represents the weights between the network input and hidden layer.
                   The first element in each row corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    output_weights: np.array representing a vector of shape (num_hidden + 1,);
                    represents the weights between the final hidden layer and the output unit.
                    The first element corresponds to the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    
    Returns
    -------
    a pair (hidden_list, output_prob), where
            hidden_list: a list of vectors of hidden layer activations at each timestep, 
                         each of shape (num_hidden,)
            output_prob: float representing the output (probability) value of the network
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(recurrent_classifier_forward)

## Activity 2.2: The backward pass in a sequence classifier

To train our recurrent neural network, we will need to implement a backward pass that calculates all the weights gradients, which are used to nudge the weights. Like in feedforward neural networks, we can do this by *back-propagating* the error of the classification (as a *local gradient*). 

The basic rules of back-propagation are the same as they are in feedfoward neural networks:  
- when we send local gradients backward, we multiply them by the weights they pass through and the activation derivatives of the units they enter;  
- when we calculate the gradients of weights between two layers, we do so by taking the outer product between local gradients at one layer and outputs of the previous layer.

But there are three major differences:  
- we have to send the errors back through *timesteps* of the recurrent hidden layer, and not just back from the output layer to the hidden layer;  
- we also have to calculate gradients for the recurrent weights $U$;  
- because we reuse the same weights multiple times in recurrence, we end up with multiple gradients for those weights, which we have to combine through addition.  

The process is laid out below:

*Note: in this notation, we're using $\mathbf{\varepsilon}$ for the local gradient / error of the output layer, and $\mathbf{\delta}_t$ for the local gradient of the hidden layer at timestep $t$. We're also using $\ast$ to represent element-wise multiplication (the `*` operator in NumPy).*

#### RNN CLASSIFIER BACKPROP STEPS

1. calculate the *local gradient* of the output layer $\mathbf{\varepsilon}$ from the *error* in $p$;  
2. calculate the *output weights gradient*, based on the hidden layer activations at the final timestep: $\Delta \mathbf{v}^T = \mathbf{\varepsilon}\mathbf{h}_n^T$;  
3. calculate the *local gradient* of the hidden layer at timestep $n$, $\mathbf{\delta}_n$:  
   1. send the local gradient backward through the output weights: $\mathbf{s}_n = \mathbf{v}\mathbf{\varepsilon}$  
   2. multiply element-wise by the *activation derivative* of the hidden layer at timestep $n$: $\mathbf{\delta}_n = g^\prime(\mathbf{h}_n) \ast \mathbf{s}_n$  
5. for each timestep $t$ counting down from $n$ to 2:  
   1. calculate the *input weights gradient* at timestep $t$: $\Delta W_t = \mathbf{\delta}_t \mathbf{x}_t^T$
   2. calculate the *recurrent weights gradient* at timestep $t$: $\Delta U_t = \mathbf{\delta}_t \mathbf{h}_{t-1}^T$  
   3. send the local gradient backward to timestep $t-1$, through the recurrent weights: $\mathbf{s}_{t-1} = U^T \mathbf{\delta}_t$  
   4. multiply element-wise by the *activation derivative* of the hidden layer at timestep $t-1$: $\mathbf{\delta}_{t-1} = g^\prime(\mathbf{h}_{t-1}) \ast \mathbf{s}_{t-1}$  
6. after back-propagating to timestep 1, calculate the *input weights gradient* at timestep 1: $\Delta W_1 = \mathbf{\delta}_1 \mathbf{x}_1^T$  
7. calculate the *overall* weights gradients by adding together the gradients from different timesteps:  
   1. for the overall *inputs weights gradient*: $\Delta W = \sum_{t=1}^{n} \Delta W_t$  
   2. for the overall *recurrent weights gradient*: $\Delta U = \sum_{t=2}^{n} \Delta U_t$

As in the forward pass, we assume that the bias terms are accounted for by the input weights $W$, so that there is no bias term in the recurrent weights $U$. We will point out how to deal with this assumption as we work through the activity.

### Task 2.2.1: Getting the weights gradients at the current timestep [2 points]

One of the key things that happens over and over in recurrent neural networks is getting the weights gradients for the input ($W$) and recurrence ($U$) weights at the current timestep ([RNN classifier backprop steps 4A-4B](#RNN-CLASSIFIER-BACKPROP-STEPS)), by multiplying the local gradient at that timestep ($\delta_t$) by the (transpose of the) input ($\mathbf{x}_t^T$) and previous hidden layer activations ($\mathbf{h}_{t-1}^T$), respectively.

Since we assume that the bias terms are accounted for by the input weights $W$, we have to make sure that we insert a value for the bias term in the input values vector ($\mathbf{x}_t^T$) when calculating the input weights gradient. We do **not** insert a value for the bias term in the previous hidden layer activations ($\mathbf{h}_{t-1}^T$) that are used in the calculation of the recurrence weights gradient, because the recurrence weights $U$ are assumed not to contain any bias terms.

Write a function `get_weights_gradients()` that gets the weights gradients for the input and recurrent weights at the current timestep. Your function should task as arguments:  
- the local gradient of the current timestep, `delta`, which is a vector of shape `(num_hidden,)`;  
- a vector of `current_input` values (of shape `(num_features,)`); and  
- a vector of `prev_hidden` activations (of shape `(num_hidden,)`). 

It should return a **tuple** with two elements, where:  
- the first element is the `input_weights_gradient` (including a bias term); and  
- the second element is the `recurrent_weights_gradient` (without a bias term) at this timestep.

*Note: you should use the `calculate_weights_gradient()` function that we have provided. This function contains a keyword argument `insert_bias_term` to control whether or not to insert a value for the bias term (which is set to `True` by default); be sure to set `insert_bias_term=False` when you are calculating the weights gradient for the recurrent weights $U$, because the recurrent weights matrix does not contain a bias term*

In [ ]:
def get_weights_gradients(delta, current_input, prev_hidden):
    """Gets the input and recurrent weights gradients for a given timestep of
    a recurrent neural network.
    
    Arguments
    ---------
    delta: np.array representing a vector of size (num_hidden,),
           containing the local gradients of the hidden layer 
           at the current timestep (t)
    current_input: np.array representing a vector of size (num_features,),
                   containing the values of input features at the current timestep (t).
                   Does not contain a value for the bias term
    prev_hidden: np.array representing a vector of size (num_hidden,);
                 represents the hidden layer activations from the previous timestep (t-1).
                 Does not contain a value for the bias term. 
    
    Returns
    -------
    a tuple (input_weights_gradient, recurrent_weights_gradient), where:
             input_weights_gradient: np.array representing a matrix of size (num_hidden, num_features + 1)
                                     which contains the gradient of the loss function with respect to the 
                                     input weights at the current timestep (t).
                                     The first element in each row corresponds to the bias term.
             recurrent_weights_gradient: np.array representing a matrix of size (num_hidden, num_hidden)
                                         which contains the gradient of the loss function with respect to 
                                         the recurrent weights at the current timestep (t).
                                         Does not contain a bias term.
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_weights_gradients)

### Task 2.2.2: Getting the local gradient at the previous timestep [2 points]

The other key thing that happens over and over in recurrent neural networks is getting the local gradient at the previous timestep, by sending the current local gradient ($\delta_t$) back through the recurrent weights ($U^T$) and multiplying it element-wise by the activation derivatives of the hidden layer at the previous timestep ($g^\prime(\mathbf{h}_{t-1})$) ([RNN classifier backprop steps 4C-4D](#RNN-CLASSIFIER-BACKPROP-STEPS)).

Since we assume that there is no bias term in the recurrent weights matrix $U$ (because it is already accounted for by the input weights $W$), we do **not** have to remove the bias term when sending the local gradient back through the recurrence weights like we do when sending it back through the output weights ($\mathbf{v}$).

Write a function `get_prev_delta()` that gets the local gradient at the previous timestep. Your function should take as arguments:  
- the local gradient of the current timestep, `delta`, which is a vector of shape `(num_hidden,)`;  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `prev_hidden` activations (of shape `(num_hidden,)`); and  
- an `activation_derivative` function (which is `relu_derivative` by default). 

It should return a vector `prev_delta` (of shape `(num_hidden,)`) containing the local gradient of the previous timestep.

*Note: you should use the `send_delta_back()` function that we have provided. This function contains a keyword argument `remove_bias_term` to control whether or not to remove the bias term from the weights matrix when sending the local gradient back through that weights matrix (which is set to `True` by default); be sure to set `remove_bias_term=False` when you are calculating the local gradient at the previous timestep, because the recurrent weights matrix does not contain a bias term*

In [ ]:
def get_prev_delta(delta, recurrent_weights, prev_hidden, 
                   activation_derivative=relu_derivative):
    """Gets the local gradient at the previous timestep.
    
    Arguments
    ---------
    delta: np.array representing a vector of size (num_hidden,),
           containing the local gradients of the hidden layer 
           at the current timestep (t)
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    prev_hidden: np.array representing a vector of size (num_hidden,);
                 represents the hidden layer activations from the previous timestep (t-1).
                 Does not contain a value for the bias term. 
    activation_derivative: function; a function that calculates the activation derivative
                           of the hidden layer (default is derivative of ReLU)
    
    Returns
    -------
    prev_delta: np.array representing a vector of shape (num_hidden,),
                containing the local gradients of the hidden layer 
                at the previous timestep (t-1)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_prev_delta)

### Task 2.2.3: Getting the overall weights gradients for input and recurrent weights [5 points]

The primary way in which backprop through time differs from regular backprop (in feedfoward neural networks) is that it accumulates weights gradients for the input and recurrent weights across timesteps ([RNN classifier backprop steps 4-6](#RNN-CLASSIFIER-BACKPROP-STEPS)). For the input weights gradient ($\Delta W$), this means adding together the $n$ components obtained at timesteps 1 through $n$ ($\Delta W_1, \cdots, \Delta W_n$). For the recurrent weights gradient ($\Delta U$), it means adding together the $(n-1)$ components obtained at timesteps 2 through $n$ ($\Delta U_2, \cdots, \Delta U_n$) (there is no component of the recurrent weights gradient at timestep 1 because there is no previous hidden layer that provides input to the hidden layer at timestep 1).

Write a function `get_overall_weights_gradients()` that propagates the hidden layer local gradient ($\delta_t$) backward in time from timestep $n$ to timestep 1, accumulating the input and recurrent weights gradients across all timesteps along the way. Your function should take as arguments:  
- a vector `delta` of the hidden layer local gradient at timestep $n$ (of shape `(num_hidden,)`);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a list `values_list` of the values vectors that are input to the model at each timestep, in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a list `prev_hidden_list` of the hidden layer activations at all *previous* timesteps, in order from timestep 1 to $n-1$ (each a vector of shape `(num_hidden,)`; and  
- an `activation_derivative` function (which is `relu_derivative` by default). 

It should return a **tuple** with two elements, where:  
- the first element is the `input_weights_gradient` (including a bias term), accumulated over timesteps 1 to $n$; and  
- the second element is the `recurrent_weights_gradient` (without a bias term), accumulated over timesteps 2 to $n$.

In the event that the input sequence has a *single* timestep, you should return `None` for the `recurrent_weights_gradient`.

**Important:** You will need to work backward through the timesteps. For timesteps from $n$ to 2, you can use your `get_weights_gradient()` function to get both the input and recurrent weights gradients at that timestep. For timestep 1, since there is no recurrent weights gradient, you will have to separately calculate the input weights gradient, using the `calculate_weights_gradient()` function with the default value of `insert_bias_term=True` (because the input weights have an entry for the bias term, but the input values vector does not).

Note: we have taken a *copy* of some of the arguments at the beginning of the code, to prevent them being changed due to mutability issues. This is important for being able to re-use functions later on.

*Hint: don't forget to pass the `activation_derivative` argument to `get_prev_delta()` when you use it, as in `get_prev_delta(..., activation_derivative=activation_derivative)`.*

In [ ]:
def get_overall_weights_gradients(delta, recurrent_weights, values_list, prev_hidden_list,
                                  activation_derivative=relu_derivative):
    """Gets the overall input and recurrent weights gradients, accumulated over all timesteps
    from the error at the final timestep.
    
    Arguments
    ---------
    delta: np.array representing a vector of size (num_hidden,), containing the 
           local gradients of the hidden layer at the final timestep (n)
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    values_list: list of n np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector
                 of input values for a particular timestep (ordered from 1 to n).
                 None of these vectors contain a value for the bias term.
    prev_hidden_list: a list of (n-1) np.arrays, each representing a vector of size (num_hidden,);
                      represents the sequence of hidden layer activations at all previous timesteps
                      from 1 to (n-1), where each entry in the list is a vector of hidden layer
                      activations for a particular timestep (ordered from 1 to n-1)
    activation_derivative: function; a function that calculates the activation derivative
                           of the hidden layer (default is derivative of ReLU)
    
    Returns
    -------
    a tuple (input_weights_gradient, recurrent_weights_gradient), where:
             input_weights_gradient: np.array representing a matrix of size (num_hidden, num_features + 1)
                                     which contains the gradient of the loss function with respect to the 
                                     input weights, accumulated over timesteps 1 to n.
                                     The first element in each row corresponds to the bias term.
             recurrent_weights_gradient: np.array representing a matrix of size (num_hidden, num_hidden)
                                         which contains the gradient of the loss function with respect to 
                                         the recurrent weights, accumulated over timesteps 2 to n.
                                         Does not contain a bias term.
    """
    # =====================================================
    # DO NOT CHANGE THIS PART OR WRITE ABOVE IT
    values_list = copy.deepcopy(values_list)
    prev_hidden_list = copy.copy(prev_hidden_list)
    # =====================================================
    
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_overall_weights_gradients)

### Task 2.2.4: The complete backward pass [3 points]

So far, we have calculated the input and recurrent weights gradients, given the local gradient at the final hidden layer (timestep $n$); this corresponds to [RNN classifier backprop steps 4-6](#RNN-CLASSIFIER-BACKPROP-STEPS). To put everything together for the complete backward pass, we need to use the error at the output layer to calculate the output weights gradient ([RNN classifier backprop steps 1-2](#RNN-CLASSIFIER-BACKPROP-STEPS)), and then propagate this error backward to derive the local gradient at the final hidden layer ([RNN classifier backprop step 3](#RNN-CLASSIFIER-BACKPROP-STEPS)). This will let us continue on to use the `get_overall_weights_gradients()` function already written.

Write a function `recurrent_classifier_backward()` which performs a complete backward pass ([RNN classifier backprop steps 1-6](#RNN-CLASSIFIER-BACKPROP-STEPS)). Your function should take as arguments:  
- the `predicted` probability output by the model for an input sequence;  
- an integer representing the `actual` class for the sequence;  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first entry represents the bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a list `values_list` of the values vectors that are input to the model at each timestep, in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a list `hidden_list` of the hidden layer activations at all timesteps, in order from timestep 1 to $n$ (each a vector of shape `(num_hidden,)`; and  
- an `activation_derivative` function (which is `relu_derivative` by default). 

It should return a **tuple** with *three* elements, where:  
- the first element is the `input_weights_gradient` (including a bias term), accumulated over timesteps 1 to $n$;  
- the second element is the `recurrent_weights_gradient` (without a bias term), accumulated over timesteps 2 to $n$; and  
- the third element is the `output_weights_gradient` (including a bias term), which is calculated only at timestep $n$.

**Important:** For the output weights gradient, you should use the `calculate_weights_gradient()` function with the default value of `insert_bias_term=True` (because the output weights have an entry for the bias term, but the hidden layer activations vector does not).

Note: we have taken a *copy* of some of the arguments at the beginning of the code, to prevent them being changed due to mutability issues. This is important for being able to re-use functions later on.

*Hint: you can use the function `crossent_loss_gradient()` to calculate the local gradient (error) at the output layer*

*Hint: when back-propagating the local gradient at the output layer to the local gradient at the hidden layer, make sure you use `activation_derivative()` to represent the activation derivative function, rather than e.g. `relu_derivative()`. Also don't forget to pass the `activation_derivative` argument to `get_overall_weights_gradient()` when you use it, as in `get_overall_weights_gradient(..., activation_derivative=activation_derivative)`.*

In [ ]:
def recurrent_classifier_backward(predicted, actual, 
                                  output_weights, recurrent_weights, 
                                  values_list, hidden_list,
                                  activation_derivative=relu_derivative):
    """Back-propagates errors through a 2-layer recurrent neural network classifier,
    to determine the gradients of the input, recurrent, and output weights.
    
    Arguments
    ---------
    predicted: float; the probability output by the classifier for a given input sequence
    actual: int; the actual class for the sequence (0 for default, 1 for alternative)
    output_weights: np.array representing a vector of shape (num_hidden + 1,);
                    represents the weights between the network's final hidden layer
                    (at timestep n) and the output layer.
                    The first element corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    values_list: list of n np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector
                 of input values for a particular timestep (ordered from 1 to n).
                 None of these vectors contain a value for the bias term.
    hidden_list: a list of n np.arrays, each representing a vector of size (num_hidden,);
                 represents the sequence of hidden layer activations at all timesteps, 
                 where each entry in the list is a vector of hidden layer activations 
                 for a particular timestep (ordered from 1 to n)
    activation_derivative: function; a function that calculates the activation derivative
                           of the hidden layer (default is derivative of ReLU)
    
    Returns
    -------
    a tuple (input_weights_gradient, recurrent_weights_gradient, output_weights_gradient), where:
             input_weights_gradient: np.array representing a matrix of size (num_hidden, num_features + 1)
                                     which contains the gradient of the loss function with respect to the 
                                     input weights, accumulated over timesteps 1 to n.
                                     The first element in each row corresponds to the bias term.
             recurrent_weights_gradient: np.array representing a matrix of size (num_hidden, num_hidden)
                                         which contains the gradient of the loss function with respect to 
                                         the recurrent weights, accumulated over timesteps 2 to n.
                                         Does not contain a bias term.
             output_weights_gradient: np.array representing a vector of shape (num_hidden + 1,),
                                      which contains the gradient of the loss function with respect to the 
                                      output weights (calculated based on the final timestep).
                                      The first element in each row corresponds to the bias term.
    """
    # =====================================================
    # DO NOT CHANGE THIS PART OR WRITE ABOVE IT
    hidden_list = copy.copy(hidden_list)
    # =====================================================
    
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(recurrent_classifier_backward)

## Activity 2.3: Training a sequence classifier

Training a sequence classifier via Stochastic Gradient Descent works just like training any other neural network classifier, except that there are more weights to consider:  

1. iterate over the training samples (in random order);  

2. for each training sample consisting of input sequence $[\mathbf{x}_1, \cdots, \mathbf{x}_n]$:  

   1. complete a forward pass using the current input weights ($W$), recurrent weights ($U$), and output weights ($\mathbf{v}^T$) to get the sequence of hidden layer activations $[\mathbf{h}_1, \cdots, \mathbf{h}_n]$, as well as the classification probability $p$ output by the network;
   
   2. complete a backward pass that compares $p$ to the *actual* class for the training sample, and uses this comparison alongside the input ($[\mathbf{x}_1, \cdots, \mathbf{x}_n]$) and hidden sequences ($[\mathbf{h}_1, \cdots, \mathbf{h}_n]$) to derive (overall) weights gradients ($\Delta W$, $\Delta U$, $\Delta \mathbf{v}^T$);  
      
   3. update each set of weights by *subtracting* a nudge based on the corresponding weights gradient and the learning rate $\eta$ *(NB: $\leftarrow$ means "is overwritten by")*:  
   
      $$W \leftarrow W - \Delta W \times \eta$$  
      $$U \leftarrow U - \Delta U \times \eta$$  
      $$\mathbf{v}^T \leftarrow \mathbf{v}^T - \Delta\mathbf{v}^T \times \eta$$  
      
3. repeat for a fixed number of epochs (entire cycles through the training data), or until convergence  

### Task 2.3.1: Training for a single epoch [3 points]

Write a function `train_epoch_classifier()` that performs steps 1-2 of the training outline above. Your function should takes as arguments:  
- a list of (`values_list`, `actual_class`) pairs representing training samples, where `values_list` is a list of the values vectors that are input to the model at each timestep, arranged in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first element corresponds to a bias term);  
- a `learning_rate`; and  
- the **name** of an activation function (a **string**, which is `"relu"` by default). 

It should iterate over the training samples, nudging the weights for each one, and then return the final weights after nudging once for every training sample.

**Note:** We have provided code that uses the name you provide for an activation function (e.g. `"relu"`) to look up corresponding *functions* to use for hidden layer activations and derivatives (e.g. `relu()` and `relu_derivative()`). These functions are stored in the variables `activation_function` and `activation_derivative`, respectively; make sure that you pass these variables as arguments to `recurrent_classifier_forward()` and `recurrent_classifier_backward()`, as in `recurrent_classifier_forward(..., activation_function=activation_function)` and `recurrent_classifier_backward(..., activation_derivative=activation_derivative)`. We have also taken a *copy* of some of the arguments at the beginning of the code, to prevent them being changed due to mutability issues. This is important for being able to re-use functions later on.

**Note:** You may assume that the training samples have already been shuffled into a random order.

*Hint: Make sure that your nudges stack up, by overwriting the weights with their new values after each training sample.*

In [ ]:
def train_epoch_classifier(train_samples, input_weights, recurrent_weights, 
                           output_weights, learning_rate, activation_name="relu"):
    """Applies a single epoch of SGD to a RNN sequence classifier,
    and returns the resultant new weights.
    
    Arguments
    ---------
    train_samples: list of pairs (values, actual_class), where each pair
                   corresponds to a single training sample for which:
                   values is a list of input values vectors (without bias term), 
                          one for each timestep; and
                   actual_class is an integer representing the actual class.
    input_weights: np.array of weights between the input and hidden layer, 
                   including a weight for the bias term.
    recurrent_weights: np.array of weights between hidden layers at successive
                       timepoints, with no weight for the bias term.
    output_weights: np.array of weights between the final hidden layer and the
                    output, including a weight for the bias term.
    learning_rate: float; the learning rate, eta
    activation_name: str; the name of an activation function, for which there is
                     a corresponding activation derivative function with the suffix
                     _derivative
                     
    Returns
    -------
    a tuple (new_input_weights, new_recurrent_weights, new_output_weights),
             where each set of weights has had a single nudge update for each
             training sample
    """
    # ===========================================================
    # DO NOT CHANGE THIS PART OR WRITE ABOVE IT
    activation_function = eval(activation_name)
    activation_derivative = eval(activation_name + "_derivative")
    input_weights = copy.copy(input_weights)
    recurrent_weights = copy.copy(recurrent_weights)
    output_weights = copy.copy(output_weights)
    # ===========================================================
    
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(train_epoch_classifier)

### Task 2.3.2: Tracking the loss [3 points]

As our classifier is trained, we expect its loss to decrease. In order to be able to assess whether this is true, we need to be able to calculate the loss over a set of (training or test) samples.

Even though the inputs here are sequences (rather than single vectors as in logistic regression and feedforward neural networks), the network is still only doing a single binary classification for each input (sequence). Therefore, loss is calculated in exactly the same way as it is in logistic regression and feedforward neural networks. For a given training sample, the network outputs a probability $p$. The binary cross-entropy loss associated with this output is:

  $$L = \begin{cases}
  -\log(p) & \text{if actual class is 1 (alternative)} \\
  -\log(1-p) & \text{if actual class is 0 (default)}
  \end{cases}
  $$
  
The overall loss of the network on a given set of (training or test) samples is the average of the losses for each individual sample.

Write a function `recurrent_classifier_loss()` that calculates the overall binary cross-entropy loss of a recurrent neural network classifier for a given set of (training or test) samples. Your function should take as arguments:  
- a list of (`values_list`, `actual_class`) pairs representing (training or test) samples, where `values_list` is a list of the values vectors that are input to the model at each timestep, arranged in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element in each row corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first element corresponds to a bias term); and  
- an activation function (which is `relu` by default).

It should return a float representing the average of the losses for each individual sample.

**Important:** `recurrent_classifier_loss()` also has an argument `eps`, which is set to a small value. You should use `eps` to adjust the value you are taking the log of when calculating the loss for a sample: if this value is lower than `eps`, you should instead replace it by `eps` prior to taking the log. This prevents numerical instability caused by the fact that `log(0)` is undefined.

*Note: you should use the function `np.log()` to take logs.*

In [ ]:
def recurrent_classifier_loss(samples, input_weights, recurrent_weights, 
                              output_weights, activation_function=relu, eps=1e-8):
    """Calculates the average loss of a RNN classifier over a set of samples.
    
    Arguments
    ---------
    samples: list of pairs (values_list, actual_class), where each pair
             corresponds to a single training sample for which:
             values_list is list of input values vectors (without bias
             term), one for each timestep;
             and actual_class is an integer representing the actual class.
    input_weights: np.array of weights between the input and hidden layer, 
                   including a weight for the bias term.
    recurrent_weights: np.array of weights between hidden layers at successive
                       timepoints, with no weight for the bias term.
    output_weights: np.array of weights between the final hidden layer and the
                    output, including a weight for the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    eps: float; the minimum value that should be used as argument to np.log(),
         to prevent numerical instabilities.
    """
    # TODO: write this function
    pass

To make sure that your function incorporates the `eps` argument correctly, run the following cell. If you haven't incorporated `eps` correctly, your function will return `inf` (and you might see a warning message). If you have incorporated `eps` correctly, your function will return ~18.4207.

In [ ]:
samples = [([np.array([2, 0]), np.array([0, 1])], 0), 
           ([np.array([1, 0]), np.array([0, 2])], 1)]
input_weights = np.array([[0, 1, 0], [0, 0, 1]])
recurrent_weights = np.array([[1, 0], [0, 1]])
output_weights = np.array([0, 100, -100])
recurrent_classifier_loss(samples, input_weights, recurrent_weights, output_weights)

To test your code, run the cell below. You should see the loss decrease from ~1.171 with original weights to ~0.756 after 1 epoch. This means training is helping!

In [ ]:
train_samples = [
    ([np.array([1, 2, 3], dtype="float64"),
      np.array([0, 1, 1], dtype="float64")],
     1),
    ([np.array([1, 1, 0], dtype="float64"),
      np.array([2, 0, 1], dtype="float64")],
     0),
    ([np.array([1, 2, 0], dtype="float64"),
      np.array([-1, 1, 0], dtype="float64")],
     1)
]
input_weights = np.array([[-1, 1, 0, 0], [-1, 0, 1, 0]], dtype="float64")
recurrent_weights = np.array([[0, 1], [-1, 1]], dtype="float64")
output_weights = np.array([1, 1, -1], dtype="float64")
learning_rate = 0.1

trained_weights = train_epoch_classifier(train_samples, input_weights, recurrent_weights, 
                                         output_weights, learning_rate)

untrained_loss = recurrent_classifier_loss(train_samples, input_weights, recurrent_weights, output_weights)
print("Loss with original weights: {}".format(untrained_loss))

trained_loss = recurrent_classifier_loss(train_samples, *trained_weights)
print("Loss after 1 epoch: {}".format(trained_loss))

## Activity 2.4: The forward pass in a sequence tagger / RNN language model

The previous activities focused on a *sequence classifier*, where a sequence is classified in its entirety. Examples of this include sentiment analysis and spam detection; we don't want to classify the *individual words*, but rather the *whole document* which is composed of a sequence of words.

An alternative structure we have seen in class is that of a *sequence tagger* or *RNN language model*. In this case, words are provided as input sequentially, one at a time, but a classification / prediction is made *after each input word* rather than waiting until all words have been provided. Examples of this include POS tagging (where we want a POS tag for each word in a sentence) and language modeling (where we want to predict the next word at each point of a sentence).

In the remaining activities, you will see how the sequence classification setup extends straightforwardly to this sequence tagging / language modeling setup. There are two main adjustments:  
1. there is an output at each timestep, rather than just at the final timestep [this is the big change]; and  
2. the classification task is multi-choice or *multinomial* (using the *softmax* function) rather than yes-no or *binary* (using the *logistic* function) [this is a smaller but still important change].

Here is a visualization of the model structure:

![RNN sequence tagger structure](https://github.com/ucsb-ling111/assignment2-giovani-gutierrez/blob/main/images/rnn_tagger.png?raw=1)

With these changes, the forward pass looks as follows:

*Note: the output weights are now a matrix $V$, and the output of the network at each timestep is now a vector of probabilities, $\mathbf{p}$*

#### RNN TAGGER CALCULATION STEPS

1. calculate the input to the hidden layer activation function at timestep 1, based on the input values at timestep 1: $\mathbf{z}_1$ = $W\mathbf{x}_1$
2. calculate the hidden layer activations at timestep 1: $\mathbf{h}_1$ = $g(\mathbf{z}_1)$  
3. calculate the input to the softmax (output) layer at timestep 1, based on the hidden layer activations: $\mathbf{y}_1 = V\mathbf{h}_1$  
4. pass these inputs through the softmax function in order to get the output probabilities at timestep 1:  
      $$\mathbf{p}_1 = \frac{\exp(\mathbf{y_1})}{\sum\exp(\mathbf{y_1})}$$
5. for each timestep $t$ from 2 to $n$:  
   1. calculate the input to the hidden layer activation function at that timestep, based on the input values at that timestep and the hidden layer activations at the previous timestep: $\mathbf{z}_t$ = $W\mathbf{x}_t + U\mathbf{h}_{t-1}$
   2. calculate the hidden layer activations at that timestep: $\mathbf{h}_t$ = $g(\mathbf{z}_t)$  
   3. calculate the input to the softmax (output) layer at that timestep, based on the hidden layer activations: $\mathbf{y}_t = V\mathbf{h}_t$  
   4. pass these inputs through the softmax function in order to get the output probabilities at that timestep:  
         $$\mathbf{p}_t = \frac{\exp(\mathbf{y_t})}{\sum\exp(\mathbf{y_t})}$$

*Note: we assume that bias terms are built into $W$ and $V$ but not $\mathbf{x}_t$ and $\mathbf{h}_t$; as before, this requires some manipulation in order to work everything out*

### Task 2.4.1: The softmax layer [2 points]

Write a function `softmax_layer()` that represents a softmax layer of a neural network. Your function should take as arguments:  
- a vector of hidden activation `values` (of shape `(num_hidden,)`, with no value for the bias term); and  
- a matrix of output `weights` (of shape `(num_outputs, num_hidden + 1)`, where the first element in each row is the bias term). 

It should return the result of multiplying the values by the weights and sending the result through a softmax function ([RNN tagger calculation steps 3-4](#RNN-TAGGER-CALCULATION-STEPS)).

**Note:** as you did in Assignment 1 for `logistic_layer()`, you will have to ensure that you insert a bias term at the beginning of `values` before you multiply it by the `weights`.

*Note: you can use the `linear()` function we have provided to do the multiplication by weights. For taking exponentials in the softmax, use `np.exp()`; for taking the sum over the exponentiated values, use `exponentiated.sum()` (where `exponentiated` is the name of a variable containing a NumPy array of exponentiated values).*

In [ ]:
def softmax_layer(values, weights):
    """Calculates the result of passing a set of values through a softmax
    layer of a neural network with given weights.
    
    Arguments
    ---------
    values: np.array representing a vector of size (num_hidden,);
            does not contain a value for the bias term
    weights: np.array representing a matrix of size (num_outputs, num_hidden + 1,);
             the first element in each row corresponds to the bias term
    
    Returns
    -------
    activations: a vector of size (num_outputs,) where the values are probabilities
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(softmax_layer)

### Task 2.4.2: Getting outputs at all timesteps [3 points]

For the RNN sequence classifier, you wrote a function `get_hidden_sequence()` that gets the hidden layer activations at all timesteps. For the RNN tagger, we will need an additional function that uses these hidden layer activations to calculate the network's output at all timesteps.

Write a function `get_output_sequence()` that calculates the outputs at each timestep of an RNN tagger, based on the hidden layer activations at those timesteps. Your function should take two arguments:  
- a list of hidden layer activation vectors `hidden_list`, containing $n$ vectors each of shape `(num_hidden,)` (without a bias term), arranged in order from timestep 1 to timestep $n$; and  
- a matrix of output `weights` of shape `(num_outputs, num_hidden + 1)` (where the first element in each row is the bias term). 

It should return a list of output vectors `output_list`, containing $n$ vectors each of shape `(num_outputs,)`, arranged in a corresponding order to `hidden_list`.

In [ ]:
def get_output_sequence(hidden_list, weights):
    """Gets the sequence of outputs corresponding to a sequence of hidden layer
    activations.
    
    Arguments
    ---------
    hidden_list: a list of vectors of hidden layer activations at each timestep, 
                 each of shape (num_hidden,);
                 these do not contain values for the bias term.
    weights: np.array representing a matrix of size (num_outputs, num_hidden + 1,);
             the first element in each row corresponds to the bias term
    
    Returns
    -------
    output_list: a list of vectors of network outputs at each timestep, 
                 each of shape (num_outputs,)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_output_sequence)

### Task 2.4.3: The complete forward pass [2 points]

In the forward pass, the network calculates the hidden layer activations and outputs at each timestep. Both of these will be used in backprop.

Write a function `recurrent_tagger_forward()` that calculates the hidden layer activations and outputs for each timestep of a 2-layer recurrent neural network, as described in the [RNN tagger calculation steps](#RNN-TAGGER-CALCULATION-STEPS). Your function should take as arguments:  
- a list of values vectors `values_list` (where the entries in the list are vectors of shape `(num_features,)`, arranged in order from timestep 1 to timestep $n$);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first element corresponds to a bias term); and  
- an `activation_function` (which is `relu` by default). 

It should return a **tuple** with two elements, where:  
- the first element is the list of hidden layer activations for each timestep; and  
- the second element is the list of outputs for each timestep.

*Hint: use the `get_hidden_sequence()` function you wrote earlier, together with the `get_output_sequence()` function you wrote above. Don't forget to pass the `activation_function` argument to `get_hidden_sequence()`, as in `get_hidden_sequence(..., activation_function=activation_function)`.*

In [ ]:
def recurrent_tagger_forward(values_list, input_weights, recurrent_weights, output_weights, 
                             activation_function=relu):
    """Sends an input sequence forward through a 2-layer recurrent neural network
    that generates an output at each timestep.
    
    Arguments
    ---------
    values_list: list of np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector
                 of input values for a particular timestep.
                 None of these vectors contain a value for the bias term.
    input_weights: np.array representing a matrix of size (num_hidden, num_features + 1);
                   represents the weights between the network input and hidden layer.
                   The first element in each row corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    output_weights: np.array representing a vector of shape (num_hidden + 1,);
                    represents the weights between the final hidden layer and the output unit.
                    The first element corresponds to the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    
    Returns
    -------
    a pair (hidden_list, output_list), where
            hidden_list: a list of vectors of hidden layer activations at each timestep, 
                         each of shape (num_hidden,)
            outputs_list: a list of vectors of output probabilities at each timestep,
                          each of shape (num_outputs,)
    """
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(recurrent_tagger_forward)

## Activity 2.5: The backward pass in a sequence tagger / RNN language model

The two differences between the RNN tagger and the RNN sequence classifier have the following implications for the backward pass:  

1. having an output at each timestep means that each timestep $\tau$ generates its own error ($\varepsilon^{\langle\tau\rangle}$), and hence its own weights gradients ($\Delta W^{\langle\tau\rangle}$, $\Delta U^{\langle\tau\rangle}$, $\Delta V^{\langle\tau\rangle}$); we need to *average* the weights gradients generated from each timestep in order to get overall weights gradients for the sequence;  
2. using a softmax output layer means that the errors at the output layer ($\varepsilon^{\langle\tau\rangle}$) form a *vector*, but are still back-propagated in exactly the same way as before.  

The backprop steps for the RNN tagger are as follows:

*Note: we are using subscripts $t$ to represent layers at timestep $t$, as before, and superscripts $\langle\tau\rangle$ to represent values that are ultimately derived from errors in the output at timestep $\tau$; thus, for example, $\delta_3^{\langle 5 \rangle}$ represents the local gradient of the hidden layer at timestep 3, based on the error in the output at timestep 5.*

*In the RNN sequence classifier we saw earlier, everything was derived from errors in the output at timestep $n$ (so you might imagine putting superscript $\langle n \rangle$ on $\varepsilon$ and all the $\delta_t$s and $\mathbf{s}_t$s in the previous RNN classifier backprop steps).*

#### RNN TAGGER BACKPROP STEPS

1. for each *target timestep* $\tau$ from 1 to $n$:  
    1. calculate the *local gradient* of the output layer $\mathbf{\varepsilon}^{\langle\tau\rangle}$ from the *error* in the output $\mathbf{p}_\tau$ at the target timestep $\tau$;  
    2. calculate the *intermediate output weights gradient* at the target timestep $\tau$, based on the hidden layer activations at timestep $\tau$: $\Delta V^{\langle\tau\rangle} = \mathbf{\varepsilon}^{\langle\tau\rangle}\mathbf{h}_\tau^T$;  
    3. calculate the *local gradient* of the hidden layer at timestep $\tau$ based on the output error at timestep $\tau$, $\mathbf{\delta}_\tau^{\langle\tau\rangle}$:  
       1. send the local gradient backward through the output weights: $\mathbf{s}_n^{\langle\tau\rangle} = V^T\mathbf{\varepsilon}^{\langle\tau\rangle}$  
       2. multiply element-wise by the *activation derivative* of the hidden layer at timestep $\tau$: $\mathbf{\delta}_\tau^{\langle\tau\rangle} = g^\prime(\mathbf{h}_\tau) \ast \mathbf{s}_\tau^{\langle\tau\rangle}$  
    5. for each timestep $t$ counting down from $\tau$ to 2:  
       1. calculate the *input weights gradient* at timestep $t$: $\Delta W_t^{\langle\tau\rangle} = \mathbf{\delta}_t^{\langle\tau\rangle} \mathbf{x}_t^T$
       2. calculate the *recurrent weights gradient* at timestep $t$: $\Delta U_t^{\langle\tau\rangle} = \mathbf{\delta}_t^{\langle\tau\rangle} \mathbf{h}_{t-1}^T$  
       3. send the local gradient backward to timestep $t-1$, through the recurrent weights: $\mathbf{s}_{t-1}^{\langle\tau\rangle} = U^T \mathbf{\delta}_t^{\langle\tau\rangle}$  
       4. multiply element-wise by the *activation derivative* of the hidden layer at timestep $t-1$: $\mathbf{\delta}_{t-1}^{\langle\tau\rangle} = g^\prime(\mathbf{h}_{t-1}) \ast \mathbf{s}_{t-1}^{\langle\tau\rangle}$  
    6. after back-propagating to timestep 1, calculate the *input weights gradient* at timestep 1: $\Delta W_1^{\langle\tau\rangle} = \mathbf{\delta}_1^{\langle\tau\rangle} \mathbf{x}_1^T$  
    7. calculate the *intermediate* weights gradients *generated from the target timestep $\tau$* by adding together the gradients calculated at different timesteps:  
       1. for the *intermediate inputs weights gradient*: $\Delta W^{\langle\tau\rangle} = \sum_{t=1}^{\tau} \Delta W_t^{\langle\tau\rangle}$  
       2. for the *intermediate recurrent weights gradient*: $\Delta U^{\langle\tau\rangle} = \sum_{t=2}^{\tau} \Delta U_t^{\langle\tau\rangle}$
2. after completing this process with all $n$ timesteps having been the *target timestep*, calculate the *overall* weights gradients *for the whole sequence* by *averaging* the intermediate weights gradients generated from each timestep:   
    1. for the *overall inputs weights gradient*: $\Delta W = \frac{1}{n}\sum_{\tau=1}^{n} \Delta W^{\langle\tau\rangle}$  
    2. for the *overall recurrent weights gradient*: $\Delta U = \frac{1}{n-1}\sum_{\tau=2}^{n} \Delta U^{\langle\tau\rangle}$  
    3. for the *overall output weights gradient*: $\Delta V = \frac{1}{n}\sum_{\tau=1}^{n} \Delta V^{\langle\tau\rangle}$  

Averaging ensures that long sequences do not dominate training just because they have more timesteps and therefore generate more intermediate weights gradients.

*Note: we have $\Delta U = \frac{1}{n-1}\sum_{\tau=2}^{n} \Delta U^{\langle\tau\rangle}$ rather than $\Delta U = \frac{1}{n}\sum_{\tau=1}^{n} \Delta U^{\langle\tau\rangle}$ because there is no intermediate weights gradient for the recurrent weights at target timestep 1, since the recurrent weights are not involved in an RNN where the input sequence has only 1 timestep.*

*Note: as before, we assume that the bias terms are accounted for by the input weights $W$, so that there is no bias term in the recurrent weights $U$.*

This looks complicated! But it's actually basically just the same thing we did for the classifier model, applied over and over with errors generated at different timesteps: step 1 here contains all of the steps that we saw in the RNN sequence classifier, if we were to assume that the sequence ends at timestep $\tau$. So if we break it down, it's not so bad!

**Note:** We will be reusing functions from the RNN sequence classifier as much as possible, to highlight the commonalities. This means that our approach will not be as efficient as it could be, because it will be calculating values anew with each target timestep when it could be reusing them. Since we aren't training the model at scale, this is OK, and we think it is better for helping you understand.

### Task 2.5.1: Getting the intermediate weights gradients at all timesteps [5 points]

The core of backprop for the RNN tagger involves getting the intermediate weights gradients for each target timestep $\tau$ ([RNN tagger backprop step 1](#RNN-TAGGER-BACKPROP-STEPS)). For our purposes, these intermediate weights gradients are calculated in the same way as the weights gradients were in the `recurrent_classifier_backward()` function you wrote earlier, treating the target timestep $\tau$ as the source of the error at the output layer. 

Write a function `get_intermediate_weights_gradients()` that calculates the intermediate weights gradients for each target timestep. Your function should take as arguments:  
- a list `outputs_list` of $n$ output vectors (of shape `(num_outputs,)`), each representing the predicted probability distribution output by the softmax layer of the model at a particular target timestep (ordered from 1 to $n$);  
- a list `actuals_list` of $n$ one-hot vectors (of shape `(num_outputs,)`), each representing the actual probabilities expected at the corresponding target timestep (where there is a 1 at the index of the actual class, and 0s elsewhere);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first entry represents the bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a list `values_list` of the $\tau$ values vectors that are input to the model at each of the $\tau$ timesteps up to and including the target timestep (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a list `hidden_list` of the $\tau$ hidden layer activations at each of the $\tau$ timesteps up to and including the target timestep (each a vector of shape `(num_hidden,)`; and  
- an `activation_derivative` function (which is `relu_derivative` by default). 

It should return a **tuple** containing **three lists**:  
- the first is a list `input_weights_gradients_list` of the $n$ intermediate input weights gradients (each including a bias term);  
- the second is a list `recurrent_weights_gradients_list` of the $n$ intermediate recurrent weights gradients (each without a bias term); and  
- the third is a list `output_weights_gradients_list` of the $n$ intermediate output weights gradients (each including a bias term). 

In each of the returned lists, the first element will be the intermediate weights gradient treating timestep 1 as the target timestep (which should be `None` for the recurrent weights gradient), the second element will be the intermediate weights gradient treating timestep 2 as the target timestep, and so on and so forth.

Note: we have taken a *copy* of some of the arguments at the beginning of the code, to prevent them being changed due to mutability issues. This is important for being able to re-use functions later on.

*Hint: your function will contain a for loop over target timesteps, where on each iteration you slice the `values_list` and `hidden_list` up to and including the target timestep, look up the predicted and actual outputs for the target timestep from the `outputs_list` and `actuals_list`, respectively, and feed all of these things into `recurrent_classifier_backward()` to get the intermediate weights gradient at that timestep. When you use `recurrent_classifier_backward()`, make sure you pass the `activation_derivative` argument, as in `recurrent_classifier_backward(..., activation_derivative=activation_derivative)`.*

*Hint: don't forget that the endpoint of a slice is not included in the slice!*

In [ ]:
def get_intermediate_weights_gradients(outputs_list, actuals_list, output_weights, 
                                       recurrent_weights, values_list, hidden_list,
                                       activation_derivative=relu_derivative):
    """Calculates the intermediate weights gradients at all timesteps by back-propagating 
    errors from each target timestep through a 2-layer recurrent neural network tagger.
    
    Arguments
    ---------
    outputs_list: list of n np.arrays of shape (num_outputs,), each representing the probability
                  distribution output by the tagger at a particular target timestep 
                  (ordered from target timestep 1 to target timestep n)
    actuals_list: list of n np.array one-hot vectors of shape (num_outputs,), each indicating
                  the actual class at a particular target timestep (ordered from target timestep
                  1 to target timestep n)
    output_weights: np.array representing a vector of shape (num_hidden + 1,); represents 
                    the weights between the network's hidden layer and the output layer.
                    The first element corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    values_list: list of n np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector 
                 of input values for a particular timestep (ordered from 1 to n).
                 None of these vectors contain a value for the bias term.
    hidden_list: a list of n np.arrays, each representing a vector of size (num_hidden,);
                 represents the sequence of hidden layer activations, where each entry in 
                 the list is a vector of hidden layer activations for a particular timestep
                 (ordered from 1 to n)
    activation_derivative: function; a function that calculates the activation derivative
                           of the hidden layer (default is derivative of ReLU)
    
    Returns
    -------
    a tuple (input_weights_gradients_list, recurrent_weights_gradients_list, output_weights_gradients_list), 
    where:
             input_weights_gradients_list: list of n np.arrays, each representing a matrix of size 
                                           (num_hidden, num_features + 1) which contains the gradient of the
                                           loss function with respect to the input weights based on the error 
                                           at a particular target timestep T (accumulated over timesteps 1 to T).
                                           The first element in each row corresponds to the bias term.
                                           The matrices in the list are ordered from target timestep 1 to n.
             recurrent_weights_gradients_list: list of n np.arrays, each representing a matrix of size 
                                               (num_hidden, num_hidden) which contains the gradient of the
                                               loss function with respect to the recurrent weights based on the error 
                                               at a particular target timestep T (accumulated over timesteps 2 to T).
                                               Does not contain a bias term.
                                               The matrices in the list are ordered from target timestep 1 to n.
             output_weights_gradients_list: list of n np.arrays, each representing a matrix of shape 
                                            (num_outputs, num_hidden + 1) which contains the gradient of the loss 
                                            function with respect to the output weights based on the error at a
                                            particular timestep T (calculated based on timestep T).
                                            The first element in each row corresponds to the bias term.
                                            The matrices in the list are ordered from target timestep 1 to n.
    """  
    # =====================================================
    # DO NOT CHANGE THIS PART OR WRITE ABOVE IT
    values_list = copy.deepcopy(values_list)
    actuals_list = copy.deepcopy(actuals_list)
    outputs_list = copy.copy(outputs_list)
    hidden_list = copy.copy(hidden_list)
    # =====================================================
    
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(get_intermediate_weights_gradients)

### Task 2.5.2: The complete backward pass [2 points]

To calculate the overall weights gradients and complete the backward pass, we need to calculate the intermediate weights gradients ([RNN tagger backprop step 1](#RNN-TAGGER-BACKPROP-STEPS)) and then average them ([RNN tagger backprop step 2](#RNN-TAGGER-BACKPROP-STEPS)).

Write a function `recurrent_tagger_backward()` that performs a complete backward pass ([RNN tagger backprop steps 1-2](#RNN-TAGGER-BACKPROP-STEPS)). Your function should take as arguments:  
- a list `outputs_list` of $n$ output vectors (of shape `(num_outputs,)`), each representing the predicted probability distribution output by the softmax layer of the model at a particular target timestep (ordered from 1 to $n$);  
- a list `actuals_list` of $n$ one-hot vectors (of shape `(num_outputs,)`), each representing the actual probabilities expected at the corresponding target timestep (where there is a 1 at the index of the actual class, and 0s elsewhere);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first entry represents the bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a list `values_list` of the $n$ values vectors that are input to the model at each timestep (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a list `hidden_list` of the $n$ hidden layer activations at each of the timesteps (each a vector of shape `(num_hidden,)`; and  
- an `activation_derivative` function (which is `relu_derivative` by default). 

It should return a **tuple** with *three* elements, where:  
- the first element is the `input_weights_gradient` (including a bias term), averaged over the intermediate inputs weights gradients from timesteps 1 to $n$;  
- the second element is the `recurrent_weights_gradient` (without a bias term), averaged over the intermediate recurrent weights gradients from timesteps 2 to $n$; and  
- the third element is the `output_weights_gradient` (including a bias term), averaged over the intermediate output weights gradients from timesteps 1 to $n$.

**Important:** be careful to exclude the `None` at the start of the `recurrent_weights_gradients_list`.

*Hint: you should be able to reuse your `get_intermediate_weights_gradients()` function.*

*Hint: you can use the `np.mean()` method to calculate the average of a list of arrays. To do so, provide the list of arrays as an argument, together with the keyword argument `axis=0`.*

In [ ]:
def recurrent_tagger_backward(outputs_list, actuals_list, output_weights, 
                              recurrent_weights, values_list, hidden_list,
                              activation_derivative=relu_derivative):
    """Calculates the overall weights gradients averaged over all timesteps in a 2-layer 
    recurrent neural network tagger.
    
    Arguments
    ---------
    outputs_list: list of n np.arrays of shape (num_outputs,), each representing the probability
                  distribution output by the tagger at a particular target timestep 
                  (ordered from target timestep 1 to target timestep n)
    actuals_list: list of n np.array one-hot vectors of shape (num_outputs,), each indicating
                  the actual class at a particular target timestep (ordered from target timestep
                  1 to target timestep n)
    output_weights: np.array representing a vector of shape (num_hidden + 1,); represents 
                    the weights between the network's hidden layer and the output layer.
                    The first element corresponds to the bias term.
    recurrent_weights: np.array representing a matrix of size (num_hidden, num_hidden);
                       represents the weights between hidden layers of adjacent timesteps.
                       Does not contain values for bias terms.
    values_list: list of n np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector 
                 of input values for a particular timestep (ordered from 1 to n).
                 None of these vectors contain a value for the bias term.
    hidden_list: a list of n np.arrays, each representing a vector of size (num_hidden,);
                 represents the sequence of hidden layer activations, where each entry in 
                 the list is a vector of hidden layer activations for a particular timestep
                 (ordered from 1 to n)
    activation_derivative: function; a function that calculates the activation derivative
                           of the hidden layer (default is derivative of ReLU)
    
    Returns
    -------
    a tuple (input_weights_gradient, recurrent_weights_gradient, output_weights_gradient), 
    where:
             input_weights_gradient: np.array representing a matrix of size (num_hidden, num_features + 1) 
                                     which contains the gradient of the loss function with respect to the input 
                                     weights, based on the average error over all timesteps 1 to n.
                                     The first element in each row corresponds to the bias term.
             recurrent_weights_gradient: np.array representing a matrix of size (num_hidden, num_hidden) 
                                         which contains the gradient of the loss function with respect to the 
                                         recurrent weights, based on the average error over timesteps 2 to n.
                                         Does not contain a value for the bias term.
             output_weights_gradient: np.array representing a matrix of size (num_outputs, num_hidden + 1) 
                                      which contains the gradient of the loss function with respect to the output 
                                      weights, based on the average error over all timesteps 1 to n.
                                      The first element in each row corresponds to the bias term.
    """  
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(recurrent_tagger_backward)

## Activity 2.6: Training a sequence tagger / RNN language model

Training a sequence tagger via Stochastic Gradient Descent works just like training a sequence classifier, with the exception that having outputs at multiple timesteps means that the overall weights gradients now come via consideration of multiple intermediate weights gradients:  

1. iterate over the training samples (in random order);  

2. for each training sample consisting of input sequence $[\mathbf{x}_1, \cdots, \mathbf{x}_n]$:  

   1. complete a forward pass using the current input weights ($W$), recurrent weights ($U$), and output weights ($\mathbf{v}^T$) to get the sequence of hidden layer activations $[\mathbf{h}_1, \cdots, \mathbf{h}_n]$ and output probability distributions $[\mathbf{p}_1, \cdots, \mathbf{p}_n]$;
   
   2. complete a backward pass that compares the sequence of outputs $[\mathbf{p}_1, \cdots, \mathbf{p}_n]$ to the sequence of *actual* classes (one-hot vectors) for the training sample, and uses this comparison alongside the input ($[\mathbf{x}_1, \cdots, \mathbf{x}_n]$) and hidden sequences ($[\mathbf{h}_1, \cdots, \mathbf{h}_n]$) to derive overall weights gradients ($\Delta W$, $\Delta U$, $\Delta V$);  
      
   3. update each set of weights by *subtracting* a nudge based on the corresponding overall weights gradient and the learning rate $\eta$ *(NB: $\leftarrow$ means "is overwritten by")*:  
   
      $$W \leftarrow W - \Delta W \times \eta$$  
      $$U \leftarrow U - \Delta U \times \eta$$  
      $$V \leftarrow V - \Delta V \times \eta$$  
      
3. repeat for a fixed number of epochs (entire cycles through the training data), or until convergence  

### Task 2.6.1: Training for a single epoch [3 points]

Write a function `train_epoch_tagger()` that performs steps 1-2 of the training outline above. Your function should takes as arguments:  
- a list of (`values_list`, `actuals_list`) pairs representing training samples, where:  
  - `values_list` is a list of the $n$ values vectors that are input to the model at each timestep, arranged in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term); and  
  - `actuals_list` is a list of $n$ one-hot vectors (of shape `(num_outputs,)`), each representing the actual probabilities expected at the corresponding target timestep (where there is a 1 at the index of the actual class, and 0s elsewhere);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a vector of `output_weights` (of shape `(num_hidden + 1,)`, where the first element corresponds to a bias term);  
- a `learning_rate`; and  
- the **name** of an activation function (a **string**, which is `"relu"` by default). 

It should iterate over the training samples, nudging the weights for each one, and then return the final weights after nudging once for every training sample.

**Note:** We have provided code that uses the name you provide for an activation function (e.g. `"relu"`) to look up corresponding *functions* to use for hidden layer activations and derivatives (e.g. `relu()` and `relu_derivative()`). These functions are stored in the variables `activation_function` and `activation_derivative`, respectively; make sure that you pass these variables as arguments to `recurrent_tagger_forward()` and `recurrent_tagger_backward()`, as in `recurrent_tagegr_forward(..., activation_function=activation_function)` and `recurrent_tagger_backward(..., activation_derivative=activation_derivative)`. We have also taken a *copy* of some of the arguments at the beginning of the code, to prevent them being changed due to mutability issues. This is important for being able to re-use functions later on.

**Note:** You may assume that the training samples have already been shuffled into a random order.

*Hint: Make sure that your nudges stack up, by overwriting the weights with their new values after each training sample.*

In [ ]:
def train_epoch_tagger(train_samples, input_weights, recurrent_weights, 
                       output_weights, learning_rate, activation_name="relu"):
    """Applies a single epoch of SGD to a RNN sequence tagger, and returns 
    the resultant new weights.
    
    Arguments
    ---------
    train_samples: list of pairs (values_list, actuals_list), where each pair
                   corresponds to a single training sample for which:
                   values_list is a list of input values vectors (without bias
                               term), one for each timestep; and
                   actuals_list is a list of actual class one-hot vectors, one
                                for each timestep
    input_weights: np.array of weights between the input and hidden layer, 
                   including a weight for the bias term.
    recurrent_weights: np.array of weights between hidden layers at successive
                       timepoints, with no weight for the bias term.
    output_weights: np.array of weights between the final hidden layer and the
                    output, including a weight for the bias term.
    learning_rate: float; the learning rate, eta
    activation_name: str; the name of an activation function, for which there is
                     a corresponding activation derivative function with the suffix
                     _derivative
                     
    Returns
    -------
    a tuple (new_input_weights, new_recurrent_weights, new_output_weights),
             where each set of weights has had a single nudge update for each
             training sample
    """
    # ===========================================================
    # DO NOT CHANGE THIS PART OR WRITE ABOVE IT
    activation_function = eval(activation_name)
    activation_derivative = eval(activation_name + "_derivative")
    input_weights = copy.copy(input_weights)
    recurrent_weights = copy.copy(recurrent_weights)
    output_weights = copy.copy(output_weights)
    # ===========================================================
    
    # TODO: write this function
    pass

To test your function, run the following code cell. 

In [ ]:
check(train_epoch_tagger)

### Task 2.6.2: Calculating the loss of a single sample [3 points]

To assess how the model is performing, we need to be able to calculate the loss over a set of (training or test) samples. Here, we will consider the loss of a single sample.

We use *time-averaged cross-entropy loss*. This loss is calculated slightly differently than the binary cross-entropy loss we saw in the RNN sequence classifier we saw above, for two reasons:  
1. the classifications that the network is doing are now *multi-choice*, not binary; and  
2. there are *multiple* classifications for a single input sequence (one per timestep).

We account for these differences as follows:

**Difference 1**

For a *multi-choice* classification, the network outputs a *vector* of probabilities, $\mathbf{p}$. We have a corresponding one-hot vector of actual probabilities we wanted the network to output, $\mathbf{a}$. The cross-entropy loss associated with this output comes from considering *only the probability of the actual class*, and taking the negative log of this probability. That is,

  $$L = -\log{p_C} \quad \text{where } C \text{ is the actual class}$$

You'll often see the cross-entropy loss written as:

  $$L = \sum_i{\left[a_i \times \left(-\log{p_i}\right)\right]}$$
  
This is equivalent to the version given above, because if $i \neq C$ then $a_i=0$, so that $\left[a_i \times \left(-\log{p_i}\right)\right] = \left[0 \times \left(-\log{p_i}\right)\right] = 0$.  
Conversely, $a_C=1$ and thus $\left[a_C \times \left(-\log{p_C}\right)\right] = \left[1 \times \left(-\log{p_C}\right)\right] = -\log{p_C}$.

This can be further written as a dot product,

  $$L = \mathbf{a}^T \mathbf{n} \quad \text{where } \mathbf{n} = -\log{\mathbf{p}}$$

**Difference 2**

For a situation where there are multiple classifications for each input sequence (difference 2), the overall loss for the input sequence is the *average* of the losses for each classification. That is, for an output $\mathbf{p}_t$ at a given timestep, we calculate its loss as above, and then we average the losses across timesteps. This ensures that the loss associated with a sequence is not based primarily on its length.

**Your task**

Write a function `tagged_sample_loss()` that calculates the time-averaged cross-entropy loss of a recurrent neural network tagger for a single (training or test) sample. Your function should take as arguments:  
- a list `values_list` of the $n$ values vectors that are input to the model at each timestep, arranged in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term);  
- a list `actuals_list` of $n$ one-hot vectors (of shape `(num_outputs,)`), each representing the actual probabilities expected at the corresponding target timestep (where there is a 1 at the index of the actual class, and 0s elsewhere);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element in each row corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a matrix of `output_weights` (of shape `(num_outputs, num_hidden + 1)`, where the first element in each row corresponds to a bias term); and  
- an `activation_function` (which is `relu` by default).

It should return a float representing the time-averaged cross-entropy loss for the sample.

**Important:** `tagged_sample_loss()` also has an argument `eps`, which is set to a small value. You should use `eps` to adjust the value you are taking the log of when calculating the loss for a sample: if this value is lower than `eps`, you should instead replace it by `eps` prior to taking the log. This prevents numerical instability caused by the fact that `log(0)` is undefined. *Hint: the `np.maximum()` method might be helpful here.*

*Note: you should use the function `np.log()` to take logs.*

In [ ]:
def tagged_sample_loss(values_list, actuals_list, input_weights, recurrent_weights, 
                       output_weights, activation_function=relu, eps=1e-8):
    """Calculates the time-averages cross-entropy loss of a RNN tagger for a given sample.
    
    Arguments
    ---------
    values_list: list of n np.arrays, each representing a vector of size (num_features,);
                 represents the input sequence, where each entry in the list is a vector 
                 of input values for a particular timestep (ordered from 1 to n).
                 None of these vectors contain a value for the bias term.
    actuals_list: list of n np.array one-hot vectors of shape (num_outputs,), each indicating
                  the actual class at a particular target timestep (ordered from target timestep
                  1 to target timestep n)
    input_weights: np.array of weights between the input and hidden layer, 
                   including a weight for the bias term.
    recurrent_weights: np.array of weights between hidden layers at successive
                       timepoints, with no weight for the bias term.
    output_weights: np.array of weights between the final hidden layer and the
                    output, including a weight for the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    eps: float; the minimum value that should be used as argument to np.log(),
         to prevent numerical instabilities.
    """
    # TODO: write this function
    pass

To make sure that your function incorporates the `eps` argument correctly, run the following cell. If you haven't incorporated `eps` correctly, your function will return `inf` (and you might see a warning message). If you have incorporated `eps` correctly, your function will return ~18.4207.

In [ ]:
values_list = [np.array([0, 1]), np.array([1, 0])]
actuals_list = [np.array([1, 0], dtype="float64"), np.array([0, 1], dtype="float64")]
input_weights = np.array([[0, 1, 0], [0, 0, 1]], dtype="float64")
recurrent_weights = np.array([[1, 1], [1, 1]], dtype="float64")
output_weights = np.array([[0, 100, 0], [0, 0, 100]], dtype="float64")

tagged_sample_loss(values_list, actuals_list, input_weights, recurrent_weights, output_weights)

To test your function, run the following code cell. 

In [ ]:
check(tagged_sample_loss)

### Task 2.6.3: Tracking the loss [3 points]

As always, the overall loss of the network on a given set of (training or test) samples is the average of the losses for each individual sample.

Write a function `recurrent_tagger_loss()` that calculates the overall cross-entropy loss of a recurrent neural network classifier for a given set of (training or test) samples. Your function should take as arguments:  
- a list of `samples` structured as (`values_list`, `actuals_list`) pairs, where:  
  - `values_list` is a list of the $n$ values vectors that are input to the model at each timestep, arranged in order from timestep 1 to $n$ (each a vector of shape `(num_features,)`, which does not contain a value for the bias term); and  
  - `actuals_list` is a list of $n$ one-hot vectors (of shape `(num_outputs,)`), each representing the actual probabilities expected at the corresponding target timestep (where there is a 1 at the index of the actual class, and 0s elsewhere);  
- a matrix of `input_weights` (of shape `(num_hidden, num_features + 1)`, where the first element in each row corresponds to a bias term);  
- a matrix of `recurrent_weights` (of shape `(num_hidden, num_hidden)`);  
- a matrix of `output_weights` (of shape `(num_outputs, num_hidden + 1)`, where the first element in each row corresponds to a bias term); and  
- an `activation_function` (which is `relu` by default).

It should return a float representing the average of the losses for each individual sample.

*Hint: you should use your `tagged_sample_loss()` function to calculate the loss of each sample, and then average these losses.*

**Important:** `recurrent_tagger_loss()` also has an argument `eps`, which is set to a small value. You should pass `eps` to your `tagged_sample_loss()` function, as in `tagged_sample_loss(..., eps=eps)`. You should also pass on the `activation_function` as in previous functions.

In [ ]:
def recurrent_tagger_loss(samples, input_weights, recurrent_weights, 
                          output_weights, activation_function=relu, eps=1e-8):
    """Calculates the time-averages cross-entropy loss of a RNN tagger for a given sample.
    
    Arguments
    ---------
    samples: list of pairs (values_list, actuals_list), where each pair
             corresponds to a single sample for which:
             values_list is a list of input values vectors (without bias
                         term), one for each timestep; and
             actuals_list is a list of actual class one-hot vectors, one
                          for each timestep
    input_weights: np.array of weights between the input and hidden layer, 
                   including a weight for the bias term.
    recurrent_weights: np.array of weights between hidden layers at successive
                       timepoints, with no weight for the bias term.
    output_weights: np.array of weights between the final hidden layer and the
                    output, including a weight for the bias term.
    activation_function: function; the activation function for the hidden layer
                         (default is ReLU)
    eps: float; the minimum value that should be used as argument to np.log(),
         to prevent numerical instabilities.
    """
    # TODO: write this function
    pass

To test your code, run the cell below. You should see the loss decrease from ~1.233 with original weights to ~1.032 after 1 epoch. This means training is helping!

In [ ]:
train_samples = [
    ([np.array([1, 2, 3], dtype="float64"),
      np.array([0, 1, 1], dtype="float64")],
     [np.array([0, 1, 0], dtype="float64"),
      np.array([1, 0, 0], dtype="float64")]),
    ([np.array([1, 1, 0], dtype="float64"),
      np.array([2, 0, 1], dtype="float64")],
     [np.array([0, 1, 0], dtype="float64"),
      np.array([0, 0, 1], dtype="float64")]),
    ([np.array([1, 2, 0], dtype="float64"),
      np.array([-1, 1, 0], dtype="float64")],
     [np.array([1, 0, 0], dtype="float64"),
      np.array([1, 0, 0], dtype="float64")])
]
input_weights = np.array([[-1, 1, 0, 0], [-1, 0, 1, 0]], dtype="float64")
recurrent_weights = np.array([[0, 1], [-1, 1]], dtype="float64")
output_weights = np.array([[1, 1, -1], [np.log(2), 0, 0], [0, 1, 0]], dtype="float64")
learning_rate = 0.1

trained_weights = train_epoch_tagger(train_samples, input_weights, recurrent_weights, 
                                     output_weights, learning_rate)

untrained_loss = recurrent_tagger_loss(train_samples, input_weights, recurrent_weights, output_weights)
print("Loss with original weights: {}".format(untrained_loss))

trained_loss = recurrent_tagger_loss(train_samples, *trained_weights)
print("Loss after 1 epoch: {}".format(trained_loss))